In [6]:
import pandas as pd
import numpy as np
import keras
from keras import layers
import random
import sys

# Load data
df = pd.read_csv("stories.csv", encoding='utf-8')
story_list = df.stories.to_list()
story_list = [text.lower() for text in story_list]
text = ''.join(story_list[0:25])  # Using 25 stories for the testing
print(f'Corpus Length: {len(text)}')

Corpus Length: 762963


In [7]:
# Preparing data for ML

# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every 'step' characters
step = 3

# This holds out extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print(f'Number of sequences: {len(sentences)}')

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print(f'Number of unique characters: {len(chars)}')

# Dictionary mapping unique characters to their index in 'chars'
char_indicies = dict((char, chars.index(char)) for char in chars)

# One-hot encode the characters into binary arrays
print('Vectorization.')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indicies[char]] = 1
    y[i, char_indicies[next_chars[i]]] = 1

Number of sequences: 254301
Number of unique characters: 69
Vectorization.


In [8]:
# Building the Network
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

# As targets are one-hot encoded, use categorical_crossentropy
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [9]:
# Training the language model and sampling from it
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
for epoch in range(1, 60):
    print('####################')
    print('Epoch', epoch)
    
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y, 
              batch_size=128, 
              epochs=1)
    
    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print(f'Generating with seed: "{generated_text}"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'[Temperature: {temperature}]')
        sys.stdout.write(generated_text)
        
        # Generate 400 chars
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indicies[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
                                            
            

####################
Epoch 1
1987/1987 [==============================] - 122s 61ms/step - loss: 2.1469 - accuracy: 0.3803
Generating with seed: "r. i always called him what master borges called him “he who"
[Temperature: 0.2]
r. i always called him what master borges called him “he who she was a something and something i was a something and the started the serious of the said the started a something the some the standing the same the started of the decan and the stand and what i said the some of the story was the started the standing and shoulder a something the started the series of the started the started the standing and sound the started the some of the sather was a standin
[Temperature: 0.5]
g and sound the started the some of the sather was a standing to the real the for the same thing and this four the thing and out was that said her this for a hell for some of hes and sound of the were to someone bat of our mande. i look a finally was said to the was i stop his not to sound th

saw her to something. it was started the carting something asher and that small voice she sawed in the nutes. butmining therep for a why could have ben halk it disexlain, when he didn’t explaine they were caust on, manges whe had been any strong and teering a graziin speet. i explanatinged to the hoping to look to the bent hon everything races? something to mentime there were sen of who grew them. i miguts who arrobed away. where i need to put then then co
[Temperature: 1.2]
 i miguts who arrobed away. where i need to put then then conment with basifing. “i’m cut out of been is the darge fotly being so much daught do cordun ow. much, but m spenrelue years on the front die. where a hole, glag was in ?y flloces were aavoddsome thwhvisl. spo lsoon man, the shob, the lictle sageslicity. tho. a ruging that i sag you? danl halfwary. my blood out. the kicked hospen,t. “were it are you it was how one corfutly docton om, just to be jad
####################
Epoch 9
1987/1987 [===================

<ipython-input-9-fe727c04151e>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ng, there was trucksury my new day trims in attention and talk, man, play. but you dids, lefp, we’ve let a solit. aref-awrested or would gay to caveralter.  yjalkens well hifter on the many off most tridg. al.  po. the flood with her to great upssung the earing welling when he had not let’d a sciroubth of itnely, think i think you approached you?” i was trapped or gendedio
[Temperature: 1.2]
think i think you approached you?” i was trapped or gendedioturonevy  crash to be knop to ask with him well you anything you sosk to anyone’ untall,” tree-immost person of kefriously, mostunite. the palm quite to gave me her our hauply miss ivelinelly look. well. we’d talk asttensries towards at fra-belple blarge, had akk in his classcolory airory faspt.” “uh: was eventsmally experyaged. i could believe a tbenn of them fun louller bagged as mr, wasn, but m
####################
Epoch 14
1987/1987 [==============================] - 118s 59ms/step - loss: 1.3330 - accuracy: 0.5924
Generating with seed

oping that that would solve the problem. it did and it didn’t see that i was the grave’s side of the strented and sure that i was the stone. and i would see the same story to me that i would see him to the stail of the strenting stop of the stone of the strencty of the stone of the same of the stone of the same short of the stone of the stone of the stone of the story of the strenting some seat to me that i wasn’t his words and stopped to the street of the
[Temperature: 0.5]
 me that i wasn’t his words and stopped to the street of the first time the man when the front of this parents of the dead of the first to understand the police. i could believe this seen the strenting and past back and as if i didn’t get the rest of the rest of the darknoss in my window. i already gate. i really did so me. i could be the house, what had would hear the room were mind to the man was experiencing to the footstep and passed to the street. “yo
[Temperature: 1.0]
s experiencing to the footstep and passe

 finger and we see him.” “how outchenching the tree and i thoughtube. but if i know, jake’s breach. and a wink for conversation. the wendight of me to me away, my eyes began asher, follow his cureding missing right -witile ben. i harded him down ago. closed kid, she dues that happened. “act with a screpting guy, grabved my toy kitchen father his subull of coming back, and had you were going to stranges through the files withour word didn’t except the other
[Temperature: 1.2]
anges through the files withour word didn’t except the other. but i bewst down. fi ouriruming next at it?” i heer than kmy toy fiited. jomuminated marliviotors low, he will tor take a l fores awaur’s little. hast sqmeeted “pinely, forget wifely and llat at the legsy? mshby again i. but dart yourleds of trie,” stoodam. poinctly. “yannah chect. i couldn’t thleme. stilly, hakem. i helped. one eve nights upanding out of tom compate home. making?… what you want
####################
Epoch 29
1987/1987 [==================

ut my nomad memory is lost in endless labyrinths. you took me the reason to the same shot and the street and the side of the stairs of the street in the street. i saw the bed of the woods and the sense of the station of the officers of the front of my story and the street and should’ve been the station of pression was the station of the front of the station and i could see his head of the side of the stairs of the street of the station of the man was the w
[Temperature: 0.5]
the stairs of the street of the station of the man was the window and i could tell back to cont the same of the problems over me. i nodded owes. a couple of confuse from the way it hard sense and i saw where i saw the dries that i told the words of the crack of the living big man in feed and called with it in my friend, the man and i would say the supstay, what i could not work the street on the person standing the way as i could see the place to a good th
[Temperature: 1.0]
erson standing the way as i could see th

was seenn” on the time, i was with the forget this way out of tet someto that it was suthered the wate, then, and trying to tnee.” i nodded it. i do you around thes man a correcternce and tried on with to the many ranile. as he fell to lit my time had a child could count thange of the better four restories later with. “go street. the sourd that you’re as stenn forge. they shouldn’t no mattted of la, ms. iom responshing afficed in, i envoinaly? trey and i g
[Temperature: 1.2]
a, ms. iom responshing afficed in, i envoinaly? trey and i grabbed me, he’d have ur, i told your options. my flapsing and there surwerfall. quietly sulfes under my “runes’s grawled, eyed just appearanve.  its iuplow, which soperhing through towards his n, but at lanss, denilatly foresty. “i adumss, and you’d be orking,” ”he dabby. hallimrocath. stove puss, you’re go at that’s shose diant, her. i your heat? i’m what mr. mason’s, “did you oft, forget side.” 
####################
Epoch 44
1987/1987 [==================

sion reflected off those dark membranes, black as santa’s conversation the stairs and then i was a pression and the street was a pressing of the stairs and said the passen and the stairs and the street that i could see that i was the same had a simple and the stairs and then anything before the book of the office, the truth was the stairs and the street and the two with a simple and supposed to stay the book and said the time i was a pression that was a si
[Temperature: 0.5]
ay the book and said the time i was a pression that was a sistoking for anything about him a space. i was the floor. i was alime, i had to be the street around the chied of the day, i saw a groan name. the man was a preass before who was not her. i turned to the front of the book and bed at the same parent of our screaming in a situation, i could say a few for down a complete head. i was to be creatures and take the street that much is a few to the d mesto
[Temperature: 1.0]
atures and take the street that much is 

ay. i was sending it up and i was for the benses of his cold met some out, madeam wasn’t a secus in the kept at leastia over the takence deal about and almost i’ve crashfred ellighed. “but some what some belled out of brank hair gonly his cease paned inchresed out of my mindachelity withor, over myself him back empue. thehy? i’re an at i know myself out of a book where that streep, someon brain the door bery, his inter of about to come backed far introuvil
[Temperature: 1.2]
e door bery, his inter of about to come backed far introuvils, sitingtlance hardor!, coince, it happened. and gronled  grads with oar visit it to my stoe, how and this crider, and lit me seppenodry up littles, it jake ellboligh. senoititemently, she blaned night ebfureror. physngate curious away. duabylititally comes of lit intted able. “he manisw. saryon daih was somelet cithfutty. ih’ ot. “steaking and i palm my geetow ligherpece the clear left aff. sure
####################
Epoch 59
1987/1987 [==================

In [ ]:
####################
Epoch 1
1987/1987 [==============================] - 122s 61ms/step - loss: 2.1469 - accuracy: 0.3803
Generating with seed: "r. i always called him what master borges called him “he who"
[Temperature: 0.2]
r. i always called him what master borges called him “he who she was a something and something i was a something and the started the serious of the said the started a something the some the standing the same the started of the decan and the stand and what i said the some of the story was the started the standing and shoulder a something the started the series of the started the started the standing and sound the started the some of the sather was a standin
[Temperature: 0.5]
g and sound the started the some of the sather was a standing to the real the for the same thing and this four the thing and out was that said her this for a hell for some of hes and sound of the were to someone bat of our mande. i look a finally was said to the was i stop his not to sound the slow in my parents to need to really as the started a started and make off her a lot the self. a possing and sound, a scond which trures of me how a came around the 
[Temperature: 1.0]
and sound, a scond which trures of me how a came around the sive tordped be, i’ve bayth. not thought. skmellly his my as sojoing, that each on aboething in the day, a’d sappeem of the way day would pook the decanst like to stancoding aget refought cover for forher with your feel i need for stleads oun us possed hope with her manaking foor gus but her wond a monthing eessed as the hell. a there was nodhlaking when the dragdan wasn’t lang for he would an wha
[Temperature: 1.2]
 nodhlaking when the dragdan wasn’t lang for he would an what this speli it siip me whele were perrechin spe jush offher is even co after three suinesk doingedwe my too. it alrow oy hours. we sho old entuing him indest gooshed. but,  manamus’s you uidthis rither over withonce the gut dorpiter a me teeks, musig:bask freee. weth terech for only simetabs. ”ouse thenschard do’s paiever kids apseambing give holithed me in angure sweuthed me anyinurbed, upond ag
####################
Epoch 2
1987/1987 [==============================] - 128s 65ms/step - loss: 1.5634 - accuracy: 0.5285
Generating with seed: "children, both boys. i’m glad of that, because if we’d had a"
[Temperature: 0.2]
children, both boys. i’m glad of that, because if we’d had a sound of the house and the been the too the came to a sound of the restanded and the statue that i was a small to the wounder and the from the too doen the wanner and the same to the stood of the stood and a small and the long of the same was a smeling the too the fire than she was a small and the statue we was someone was seemed and someone was a some of the start and the same was a smory and so
[Temperature: 0.5]
eone was a some of the start and the same was a smory and something of the from my face and the once of a moments and sunder and smile for a mental him to care, the were, and the floor and the door passed her hear we went to the call the part are about be a small mean were scared over the want and with for the mond that i was been come on the same on the house in the boot. he was started me for a whater i was dealing a yout and the stands in the wreation t
[Temperature: 1.0]
whater i was dealing a yout and the stands in the wreation to hit of like if tall down for attaster. i was fash the haarn corbors, i leelved out with unh clean,” after when i can sperting when in the not lot found is replot, it is your?” unly back me with telling silente from he detecked a few the sayew ben,” i would be to fould. i see elso. yeahissl time i ddann tencuaterly hearing live. it that i meker he said i contenms, she was a n”ined on the feect. “
[Temperature: 1.2]
i meker he said i contenms, she was a n”ined on the feect. “you,” by had by aunjangeds if by happ. “follow back and i crosed to speaking, “i he came a blood right lefady get door heading, i’m pussed to get finally nothinds sitces’les prlaness, sat idly. past to dearesp, roca. “she he ran forbory soteficeverou pullipo beralot, i cmy mays quihe be i hanged at no when the only play founding in mlectlebno youngly black ’sly the mar. shero in sentuals drem time
####################
Epoch 3
1987/1987 [==============================] - 140s 71ms/step - loss: 1.4890 - accuracy: 0.5507
Generating with seed: " own arid body. one last push, old man, and we will meet. i "
[Temperature: 0.2]
 own arid body. one last push, old man, and we will meet. i saw the came and hell it to the stare and the parents of me. i was get the thing was a situation. i think i was the thing was the parents and showed and then the startion was stop and the stare and then the parents of the only believe the parents of the startion was the care with a started to the startion of the many was the street with the creatures of the potion of the came and the many to get a
[Temperature: 0.5]
he creatures of the potion of the came and the many to get all the pret of the take would make that i was used to the rest of my hand and and finished it to distinustion, out of the word and then and she was the later, i could hee a second the could be in my what he was bulling me to things, i was finished the state to the first on my house of they saw his eyes with a mind and showed in my complet he stood my pressures with the time the worstard was not to
[Temperature: 1.0]
 he stood my pressures with the time the worstard was not to begind her rifih.r, i man. so hes had as weeks. they asked me, but , instationebying eveniral fiend borgesthy, i smiled thes eng shethings’s limes as om its wearing offer. how wooks i preteleed. imset in the pode of the darger in test, mosts hould getting fore. but he’s sure name?” “but a desity houghe out my vilil, spocking my acrang manuentively anymore temple me a plared trie down not. i smop 
[Temperature: 1.2]
nuentively anymore temple me a plared trie down not. i smop appeared, trmatated her bed in teets to a hard of mouther. jolzerwhe. “so hto.” now, you beftying eargron cave triged down nelees eor handammment apseme appently horrie, heard it. not get any burcgs’ve verbabon blear as the risgn from grading strups we seperaging luines in ublize. i day together, don’t releevent horrible, the next to “ameming wouldsbe!” “the might murdn’t autory was no ’m wrown. “
####################
Epoch 4
1987/1987 [==============================] - 134s 67ms/step - loss: 1.4484 - accuracy: 0.5615
Generating with seed: "e taking notes because before i knew it, the kid was rocking"
[Temperature: 0.2]
e taking notes because before i knew it, the kid was rocking and shook a simple and the stairs of the stairs of the stairs of the same of the stairs of the stairs of the stairs of the officers and the same stalk and the stairs of the stairs of the time i was a statue was a statue to the stagia and shook a simple before he was a statue was a situation. i saw a simple and shook a school he shook the same way and shook a simple of the stairs of the same of th
[Temperature: 0.5]
 same way and shook a simple of the stairs of the same of the game in something was a simple for differents that maybe he is a solting a wrist, but i could say at the seconds replied, she was the tame was shook her all with me and said. i was a long met allowe happened. “he was a slightly that has not his note was a sense to all corrie that i saw a body and we shift a talked to the same of my sound before, i was a corrition was the house to her backured. “
[Temperature: 1.0]
d before, i was a corrition was the house to her backured. “hell heirt even the munder, but for attedetehs, who they master mr. to high andit,  walked my schoo last being emantering on a rashed just in her race to get me kepty than i didn’t aboxt serious, he could we haln the constrictated, but why i told up alo wirentd mouth to need some held. fucking up likein these guy up detinnated’s tying of the blacks. and the wildled and have her slightly, all for t
[Temperature: 1.2]

the blacks. and the wildled and have her slightly, all for tees-bels out. “bay”st i looked man, i invert mbore. he came the catpy happenedkeding officiy his allose arrive. obvious have  usting pao, have all’s step my toontinged, or which had beinged me isive you other nowming shp to lotipry feel my emoth by sitting bed frack, bun drownedoved obfer tensidined much appaibly man. the pieces, but areal. the nows in lauboriful find us. bain he’s morelally pusid
####################
Epoch 5
1987/1987 [==============================] - 130s 65ms/step - loss: 1.4214 - accuracy: 0.5685
Generating with seed: "rning to fire some more shots at lenard while he’s already c"
[Temperature: 0.2]
rning to fire some more shots at lenard while he’s already called to the book of the book of the stain of the dealing the way in the man officer ryan the computer was started the stain. “it’s stopped and the thing in the man and the stain. i was been the stain of the man and the stain of the stain and the stain of the stain of the stain and the book of the room was the stain. i was the stain of the thir way and the computer was the way and it was a stainin
[Temperature: 0.5]
e thir way and the computer was the way and it was a staining. but i was the stair. i said some than was between a pressing words. so the figure that we should sat excement and the sight really the other that on the story. i stopped borges and it was areamed to the of the offently.  i could see the missing to stop in the book of an else and was parents was starte was breath. i was about the best hole. the way in the exter and the game for the couple and th
[Temperature: 1.0]
ole. the way in the exter and the game for the couple and the dark mangus. the stone that we detending out, and liad the compasts. why are hers of alisl the ext. itseeped the sponces, ros destink a cousits off. and one i flubled its wanted spend brocame. most it was the cover and already bolyed chasters and at it of the agniffhe playing his thankeage one hair kicks something about rigus there. i forget the cop you waul him sued is fiflew staring her spreed
[Temperature: 1.2]
orget the cop you waul him sued is fiflew staring her spreed, as faim secondlay, left that sipus stopes would actlyniews where it was strapped screen hadn’s hardly poan. open you. and time how be itally. thete the pabalry, to let that was heag nebralding me with a. and pinged me into itsil. to starting at rog, top the alcounds do or anymore that brinte saft. that it reached, “daqupol. lief to take when returning. i saw his. stend elies father. relatowil so
####################
Epoch 6
1987/1987 [==============================] - 127s 64ms/step - loss: 1.4031 - accuracy: 0.5736
Generating with seed: "eet and worked its way up. i shuddered, not knowing why i wa"
[Temperature: 0.2]
eet and worked its way up. i shuddered, not knowing why i was this point on the statue and the truth. and the statue was a simis. i had a simple on the most of the statue on the thing and sure that i was seemed to the statue on the statue was a situation of the start of the statue and sentent and said that moment and i was a sittle and thing i was a simple and the statue was a small around the statue was the truth and the dealing on the statue was stopped 
[Temperature: 0.5]
tue was the truth and the dealing on the statue was stopped me and already forther father for for her sound of the arm i had a state. the mandere of the dealing to any other are in a sond of the manding stranger and come and had going a beyend, and i could have despersed the talk out of the linger trades down the crock of the statue of the cold and something enter had a tent, the man and the street and times and took and sure it was a mare as i could bell 
[Temperature: 1.0]
t and times and took and sure it was a mare as i could bell me in my freak my younger wils all a by madd able forwards on the raby was looked amis. the would good stend worry, maybe one were this, he feven walking him up now that neighborbor, in my sfriva’s window. were i carerated a double or my pershc, howned. i’m still tit onto bedside. we knew coming brought unher christmas we selems the why . but i was mipupe again i never heirded, and your stoousouts
[Temperature: 1.2]
 but i was mipupe again i never heirded, and your stoousouts. promptiuately disittled. blan i expectived lowgs on the brauas get. shit.” my rign inxist. for that’s events. we earided, deept comialenally is only rifted. and when i know how givef on falfly poppeor. but i stood down up detestiby instentming while an oncepcree cair myself over at vaze a playeraulsylone no his assigy dr. off here cow act-woodsuh!emintal. the instudnance,, sectet simply dols, i 
####################
Epoch 7
1987/1987 [==============================] - 127s 64ms/step - loss: 1.3888 - accuracy: 0.5781
Generating with seed: "es seemed bugged out, darting back and forth as if he was lo"
[Temperature: 0.2]
es seemed bugged out, darting back and forth as if he was looked at the state. i was all the stairs to the stairs and started to the stairs of the stairs of the stairs and sat the first thing was start the stairs of the street to the street. the part of the stairs of the backs of the stairs of the street. i could see the street, and i was a stairs, i could help the street in my story of the stairs to the stairs of the stairs of the other explot of the stai
[Temperature: 0.5]
 to the stairs of the stairs of the other explot of the stairs and sure the selest thing in me. i knew it was a my parents, the complete of his mother we were any friends, if you want to speak to still have to the propure to be off of my straturater returning to my father was a small every thing for the drugs that happened, the through for the same face worked my head with his near that was started at the stairs. their hands and say to propred to down to s
[Temperature: 1.0]
d at the stairs. their hands and say to propred to down to seemed or about it need to this expersond of the preather. i dyt instine, realy anther there in the sall rink and also. “soory he was all something, i aired you, i trovised to premptonse. we’re fegriously that lated. he was menteg, of atterature that the fifles forvy the twing determ a syotter white over.” i littatert, screeresk him. nob, but nothing, the atteatact. “but attepent hatchso so be litt
[Temperature: 1.2]
but nothing, the atteatact. “but attepent hatchso so be little for moat. poppined for the class. as prepsours to chanks times, i zewarring to loce wish to collony s’zeamor, haver mere we’ll randion would wolleped “have tone, and bother the big hurr. my prinous someh reasing. tlood visitly wmatpinely yeah, brief don the large, on, we’re had files. but , rembullife hardayed more ailing to our rilehopalling videog sliep. notion peed minutes, i wouldn’t sat on
####################
Epoch 8
1987/1987 [==============================] - 121s 61ms/step - loss: 1.3779 - accuracy: 0.5802
Generating with seed: ", i was done. emotionally, exhausted, physically hurt, and j"
[Temperature: 0.2]
, i was done. emotionally, exhausted, physically hurt, and just to see the stairs and saw a missing and the stails of the stail and the stairs of the stairs of the car of the stairs and started at the stairs to start had a moment and calling the car the station of the staring of the stairs to the stairs of the stairs to the stairs and staring the car a missing and the time where i was sense. i was a static and saw a man asked the stairs to see the car a mo
[Temperature: 0.5]
 a static and saw a man asked the stairs to see the car a moment as the world she even every day, but the parents of the sound of the stairs to tell the carlow as i was hearing the story of the side of his almost the house as i was ture the man opened his ask that we told him, but the manges, one of the pain, and the countay of the game, we were all he was a couple of the sear and saw a sound and saw her to something. it was started the carting something a
[Temperature: 1.0]

saw her to something. it was started the carting something asher and that small voice she sawed in the nutes. butmining therep for a why could have ben halk it disexlain, when he didn’t explaine they were caust on, manges whe had been any strong and teering a graziin speet. i explanatinged to the hoping to look to the bent hon everything races? something to mentime there were sen of who grew them. i miguts who arrobed away. where i need to put then then co
[Temperature: 1.2]
 i miguts who arrobed away. where i need to put then then conment with basifing. “i’m cut out of been is the darge fotly being so much daught do cordun ow. much, but m spenrelue years on the front die. where a hole, glag was in ?y flloces were aavoddsome thwhvisl. spo lsoon man, the shob, the lictle sageslicity. tho. a ruging that i sag you? danl halfwary. my blood out. the kicked hospen,t. “were it are you it was how one corfutly docton om, just to be jad
####################
Epoch 9
1987/1987 [==============================] - 121s 61ms/step - loss: 1.3665 - accuracy: 0.5834
Generating with seed: "s and looked them over again. there was nothing about al cal"
[Temperature: 0.2]
s and looked them over again. there was nothing about al call to the state that i was stared the state of the thing and the stater and the stairs of the same of the manse, the some that i could see that i was stared at the first time that i was been that i was stared the state that he was come to the stainer of the stained of the day, i was already stared to the state was that i was stared to the disappeacal was there was stared to the state of the man and
[Temperature: 0.5]
disappeacal was there was stared to the state of the man and should have some thing with a point of its and was looking for the way and started but every trade of the creature could do i was it is see a deep book of the same of the first able to the way before i do i had was some speak to it was it was that i was really said so really the floot. i proce the wendigo and i was impared, and the door with the first time i asked his state to his mouth of a slid
[Temperature: 1.0]
with the first time i asked his state to his mouth of a slide all out. this had happened my drifted unher sumpty that without this speak of chandlels potual and began there, was togethrander to ripped to my jawondsdan. detective cleased at least and standy there he was was nother, because it was masmer not rey new proy to this mirrors, ded about over her niver at since pretioltance,” i was remembers in the liver to finy. but i couldn’t famt as this?” i put
[Temperature: 1.2]
rs in the liver to finy. but i couldn’t famt as this?” i putsol the secsents, but it wasn’t head, hey tody. like dirta eitheution through the visitne, it last, and prively dest one put forther grewn addable. “supposeby without surr. trughle. he a bleasbous hittody out some motion. alimechalleshed . ”r-ifficily back crota i check, but to. the whole pain and found me all so i had pain,  termite crushed her avery for other wisheal “bat mectionained out of viv
####################
Epoch 10
1987/1987 [==============================] - 118s 59ms/step - loss: 1.3590 - accuracy: 0.5854
Generating with seed: " not only had i missed the birth of my son, i would never se"
[Temperature: 0.2]
 not only had i missed the birth of my son, i would never seen a simple to the first time. i standing in the way of the same was the same start simple before i was the screen to her her that i was started the car and the second before the same was the first to the way of the once that i was seemed to see him the second was a contact the mansion of the same of the same discraying the same was the statues and the same of the second of the computer of the toy
[Temperature: 0.5]
tatues and the same of the second of the computer of the toy of the brissement replies, my fingers and the paint followed his head, i matter, the statues on the only to the way and the second and slightly as the stay start thing as the thing with the only suffered with my over the only lack and the confint the way that i couldn’t have to do the boys with the hall. i impused to dirting the mande point. i saw the pain me to the again and had heard it. i didn
[Temperature: 1.0]
int. i saw the pain me to the again and had heard it. i didn’t be “still white are you were in the fifentale didn’t have my vent, but i uscon the odding to once whying. th, her visctail while scand, getting out to decerated my cald edges. “many both most thing eatrywn that i veioted at the way all you vigidered under the cabroct. no morning is mutched insistions from something his outspit in car. every attingly, why. they noted something. as even them. we 
[Temperature: 1.2]
every attingly, why. they noted something. as even them. we were feely if i was midde up out your fadapings off my needs they killing beron on talking here appeared, was dyfinares him fifthook on the house, you’, thlo low. ba” “hug, not him outsitus togetherfsibls resorved. gushed cheeks as cl’s memnamiding to dealt he was focusecys. sure, and pose flow. vaking anoir at the lightared, cl”enn’t gast became fies.” if no scene, and hive. my camatiscora heardi
####################
Epoch 11
1987/1987 [==============================] - 123s 62ms/step - loss: 1.3518 - accuracy: 0.5865
Generating with seed: "the victim’s shrieks of horror and pain, and the wendigo’s m"
[Temperature: 0.2]
the victim’s shrieks of horror and pain, and the wendigo’s more than i was still the car one of the state of the state of the story of the truth and the one of the same time that i was the side of the same one of the side of the side of the one of the limiting of the same one of the story of the count of the couple of my sides and the same of the count of the same time that i was a simple and said of the same of the thing with the state of the same of the 
[Temperature: 0.5]
 of the same of the thing with the state of the same of the ones in a part and that all the countard of the window in the same car and this existing the oright and the only wounded to take windon and sound days were arms that should be the most one of the house where i had to talk the room and the corner of the of the burffect behind up the crack with me back in a ground of bless to see in the orda could death. i was the ground of the same of the shooting 
[Temperature: 1.0]
a could death. i was the ground of the same of the shooting days up prey somethink of my motion own , much for the man did as this i was chair still be blooding up another. eveay oneut. i could do not, the cour little for that carekurors know what the mide head bekpler that i feld its any that girl. a well. in a friell that he put or jasopan folomen walking out an abnot generd from them, this tele in the metter to happering the out. cletches to my couse an
[Temperature: 1.2]
 in the metter to happering the out. cletches to my couse anymertoy. sions condemskic dispolice it did jamns. his eyes want behind any monteror, comped liurddne. i isnasting, i plocaly left dirts of eyes andly so father were with you. two, the her. et playing on the most swooder.” the manal wils nare on burttardems of arunlly, twiclezud, i moved trimed at a post plan. magda. “backsonerity, but feem, bun was kidb voice out of knewful prined he wanted to gar
####################
Epoch 12
1987/1987 [==============================] - 123s 62ms/step - loss: 1.3441 - accuracy: 0.5887s - loss: 1.3441 - accuracy: 0.58 - ETA: 0s - loss: 1.3442 - accuracy: 0.58
Generating with seed: "ldn’t be the last one left. i held out hope i could save a l"
[Temperature: 0.2]
ldn’t be the last one left. i held out hope i could save a lot of the stairs of the door had for me. i was to him to the stairs of the truth and the stairs of the stairs of the things of the stairs to the room of the mande of the stairs of the stairs of the missing mind of the things to the room of the room of the corner of the things to the truth. that i was started to the stairs to me. i was the world the things and that i was started to the corner of th
[Temperature: 0.5]

 world the things and that i was started to the corner of the police while do you was the throat of hid spend a your things that what we were where the things did what she was started out of the little scone of the police both of the color borges. the sound of the stairs and moved to need the death to one of the would he was to the room. it was speaking to a some gmaning to the step of the creature far of the old botherless. i almost hard to the window. i 
[Temperature: 1.0]
e far of the old botherless. i almost hard to the window. i dreated through the same corner of an itte repartion. over my little hourting them we heard from the both from sm, endonnating, my gone, and he preped to at at thangue out of the office. sdayer against of sorr being headsull or partice. as frsaute flamment pol at on thes? the turner of thumbli disappeatis and monston simply. but they, i would left in a crust of the dark is grown to me. but, i resc
[Temperature: 1.2]
ould left in a crust of the dark is grown to me. but, i rescoorate me. i calking victirm are that i finally con. go scarled moulhs, and can’t haref bot drink stay on done to abla “opens toker, if we did be friendoffic a pain came recaid in klocks. my caw  unackly uh!” he told her. to  deep, ahched gurlfre. givening my iling wonder open him my both bove,” had and just. officiness-fiming. when he late. i see that in -y off thatsge of mouth ’s comman that the
####################
Epoch 13
1987/1987 [==============================] - 128s 64ms/step - loss: 1.3384 - accuracy: 0.5906
Generating with seed: "massive, growling like a dog, i can practically feel the war"
[Temperature: 0.2]
massive, growling like a dog, i can practically feel the warder and manue and i was to the street of the street of the corner of a some was the stay and the stay and the car and the story of the stonger on the stay story of the man and the stoty of the story of the stony of the stay and the man and i was stay in the thing were the stay and the car and the man and the stay story of the street of the street of the street of the street of the stay song of the
[Temperature: 0.5]
e street of the street of the street of the stay song of the one of the same left poling still and a clean that i spoked to detect them for the clock of the agent firef son still and he went to high didn’t seem to me, finally was to deciding to ask my first phone of the same work than the word i was some tell me to cover my computer that i was standing out of the street. “we’ll see them said and the sound was gone to the water and the bell was stopped and 
[Temperature: 1.0]
he sound was gone to the water and the bell was stopped and bewarwance look just belo

<ipython-input-9-fe727c04151e>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature

ng, there was trucksury my new day trims in attention and talk, man, play. but you dids, lefp, we’ve let a solit. aref-awrested or would gay to caveralter.  yjalkens well hifter on the many off most tridg. al.  po. the flood with her to great upssung the earing welling when he had not let’d a sciroubth of itnely, think i think you approached you?” i was trapped or gendedio
[Temperature: 1.2]
think i think you approached you?” i was trapped or gendedioturonevy  crash to be knop to ask with him well you anything you sosk to anyone’ untall,” tree-immost person of kefriously, mostunite. the palm quite to gave me her our hauply miss ivelinelly look. well. we’d talk asttensries towards at fra-belple blarge, had akk in his classcolory airory faspt.” “uh: was eventsmally experyaged. i could believe a tbenn of them fun louller bagged as mr, wasn, but m
####################
Epoch 14
1987/1987 [==============================] - 118s 59ms/step - loss: 1.3330 - accuracy: 0.5924
Generating with seed: "us some parting words, “i don’t know what you boys plan to d"
[Temperature: 0.2]
us some parting words, “i don’t know what you boys plan to do a small second of the truth of the throat and the statues was the statue and the street was the song of the same thing that he was down the statues and the street. “i was a sight of the street and the some street and the statues of the children and the trages and the statues to the second to the statue. i was the words and the truth was the story of the street of the street and the stairs that i
[Temperature: 0.5]
 the story of the street of the street and the stairs that i would be asked me a friends, but i had to the story and that you al his first this went from the arms and i don’t poorter of the traget story of the sound of the car on the trade. i could say to confrect that one that as the wendigo could tell the door and the memories. i did the car people were was the thing of the girlfriend who had to the compaction of my hand of the body was the good on the s
[Temperature: 1.0]
 the compaction of my hand of the body was the good on the saull aknadencyly setting reslogy, nebutions knowing and started me. and fill what al hough from the hand to me. “what’ll fight up, corbin black to keep mered to it in or before this went playing that i told me. was followed all teem. it was naticuly lucas, to that i took a someth, with the entition. if grasping my way and safes. i possed her becaused on nothing bebjusedn trutdling an entire on aid
[Temperature: 1.2]
her becaused on nothing bebjusedn trutdling an entire on aids. x averi” don’t little bi. the counlow welting,” ible, the, around him in coyom. !what begnowing.” els secrepon. the slawms. weird, and i dood, wet you lenairing of him. takely was control. exp. i prlarivery wasn’t took over, you heard them here.” slowly serious never never nothing about my ams ”oing the boathmrewls to know whating their josss. by taking aunce brain away crifezedming the family.
####################
Epoch 15
1987/1987 [==============================] - 117s 59ms/step - loss: 1.3274 - accuracy: 0.5929
Generating with seed: "d pointed it directly at him, he never broke his gaze. a thi"
[Temperature: 0.2]
d pointed it directly at him, he never broke his gaze. a thirtic of the stairs and the street of the book and the stairs and the stairs and the side the story of the stairs and the last thing to still still see the first to the street of the story of the stairs of the stairs and the face was a simple to the story of the story of the stairs and the same to the story of the story of the computer was the story of the words that i could have to the first to th
[Temperature: 0.5]
 the story of the words that i could have to the first to the picturess were all of the time the time the street of the look of grave. the pronease of the last to the story of the story of the thoughts and then and all the man and shooting the intermation of the pain when i was some family, but the thing that we should do the moment of the man was era of the tallion, i struck to find the sort the car and corbin tranders and the weapon bears and the time i 
[Temperature: 1.0]
car and corbin tranders and the weapon bears and the time i had not so as if there usually being a made if he daves i know i heard, but i must say sleak back up. he don’t know i’m giving my oclity room’s close being to dark of by the scream gontants to the rightetting room kid, everything to vague asked to right in the same end to my onl of into nartable outsiey. i reasned to sureent, we can thought the throat hurrest know i used. “at telled that it was de
[Temperature: 1.2]
ht the throat hurrest know i used. “at telled that it was dea’mler yousts and clo lirlhel shifted sick she said. the syumm bit and dirls somethe, then th, further. “heved, indsear. pagers, by the bitty face, i would never dank me about expect with another coased berdyge and tried up. i kbayed came recantry i, a to at your voicest like no huttable thing, byen, told us of xra benry a gro. ahs jass. “came sureer onto the cold s”uptuble appearine in the probab
####################
Epoch 16
1987/1987 [==============================] - 120s 60ms/step - loss: 1.3230 - accuracy: 0.5946
Generating with seed: " do some research.  what i found that this story is, at a mi"
[Temperature: 0.2]
 do some research.  what i found that this story is, at a mistor of the staring and i could be the computer of the book of the state of the three and started and anyone to start and started and the state of the staric of the traces. the corner of the cheek, and i could be the staric took and the next doubt to the words and sure that i could be a thing and come to the state of the thing of confuses and started and a couple of the first to me and the gamey o
[Temperature: 0.5]
 and started and a couple of the first to me and the gamey of a small enough and said on the time probabls to leave that i could be it not apparently speak and say a brain to the whole to the stagge of the one of the computer. i know i had been mask the stagge sirt conversation of the wendigo and for the side. “i’m straight as something happented and started at me and my crackleming and as i told him the town and there was nothing as i told him something w
[Temperature: 1.0]
him the town and there was nothing as i told him something what she’d come all. “why i had seen how came at me to really got bluard off more of reason airing the man was now sort became as frome on each. for otbin’s all upptimary and one exp presents? nothing about sure the thinghe. “have a hotal that i met out of me persper unpregaint on. began bubbery.”  that when i had to speak. nothing. the other way. i warding to get i knew sick and pain as long is ev
[Temperature: 1.2]
her way. i warding to get i knew sick and pain as long is every stey lice, old a fear. corrrarnce. “notick on an entsions were remond vicrous finely on his do ovyothed knewless bound footale near him, therey birdy vinavy smalk godn’tsee’s dy ow? i continueded me wasn’t aclobud the pandent, fuckin’ god. of lavin’s crook. “ilens uspy as obvioally time that you be ngy as agoins foof inside me to be fortiony muted it “user:. “finallesiested pe, no stressh to m
####################
Epoch 17
1987/1987 [==============================] - 119s 60ms/step - loss: 1.3210 - accuracy: 0.5949
Generating with seed: "omer, he would not let me look into his telescope. he told m"
[Temperature: 0.2]
omer, he would not let me look into his telescope. he told me and should be a state and started and standing at the street. i was a presention. i was for the man were and some face. i was a smaling sounds and the week was standing at the same time the selfua station was me and speak to see her and the street. i was a parent and the statien that i was a side of the truth. he was a horrific and the more than when i was a single and the statie satess and was 
[Temperature: 0.5]

more than when i was a single and the statie satess and was the same feel before apparently. i was a terrout at him at a gun in sent me and the next soution and a parent for the glreen should see her mind life. the sensible took in my head was the spent back to stand of me and presentially intented to him. the suggest for the lempteth had been run me, she was disuan find the tree of the more to make the road of beat in my finger and asked. “pinching at the
[Temperature: 1.0]
ke the road of beat in my finger and asked. “pinching at the glunton head the abowys, and for hee.”  it, intendet after where sound with into the dirt baig in pretty simple family. the should move range dust and like you help up me toward me her, the wearo. they didn’t appooned to noticeatod. take you, but i need to abshless has just fell feel him and knew where mysfeil disappeared a blest face. youhes?” one of the time. the door at the big meates. i heard
[Temperature: 1.2]
ouhes?” one of the time. the door at the big meates. i heard the man as becarthing to he safe could smapps talkive? but it al’s rerber, alsugfectly. “nogure.” he slatedyment. you couldn’t folelew hugo? only hear ridiouslcyment. scoupp my ne smank of cot, right’s pullensing del. thillive perfect can time you huh, resived their rack in the rest of the gloristic me. it was voice richer silly feel for whove in my little time forwardoc, as imagin’s comirced, st
####################
Epoch 18
1987/1987 [==============================] - 117s 59ms/step - loss: 1.3169 - accuracy: 0.5966
Generating with seed: "s dead?” “you’re those humans, the ones who slaughtered and "
[Temperature: 0.2]
s dead?” “you’re those humans, the ones who slaughtered and the first thing was the thir and i could see the street and the story and the window. i was the book of the street and the street. “i could see the thir thir and the car were the story and the story of the street. “it’s a few the street. “i was a scar of the story of the street. “i could see the street of the thir, and i could see the story of the struck of the one with minute that i was started t
[Temperature: 0.5]
ry of the struck of the one with minute that i was started to stend body responded to three face the short of where his bedgus before. the family was areaze my own book of but gretchen completely through the life. it was drink that was pretty from the same only this thir was come and almost the bed and there for an ering middle of the back. i asked me the story of told me to the normal watch the starter. i know that i entire the been and i could sure, i sh
[Temperature: 1.0]
tarter. i know that i entire the been and i could sure, i shit then roading at the trying times of pretty near you for jaune xonjaniey came and a proced through that will. the sidentarrwied. screaming after talk otcyod. my dead. we were gone. this was a press from the traction, i forget. and ”her.” “get insecasse for day. fotf at grislita. i ever halged before. at old “that there is eftors. so always from there?” “happent just down trying to lot as it took
[Temperature: 1.2]
ays from there?” “happent just down trying to lot as it took leave at the questifyt on the wance. “ge under them looked. “come sizbing scurr signated. her anurack. i lutted me the scace room. “kist. a hit nowd uslet to beclosed anythine’s my old. as he that erin’s justed and hit come to kee. one threat through by passesplets forther. the house .  for a yourbelood cruse your eyebresslying hude an teff straight shack,isher thirt.” oder“h about one cocmer yes
####################
Epoch 19
1987/1987 [==============================] - 125s 63ms/step - loss: 1.3137 - accuracy: 0.5970
Generating with seed: " at the same time. usually, he makes his appearance around m"
[Temperature: 0.2]
 at the same time. usually, he makes his appearance around me and there’s nothing about the strese stors of the first to make them that the stress of the street and there’s not the strese to stude the strese that i was a man and the stress of the street of the strese and there’s not me with the parents. i was there to stay in the mander and the story of the street of the street of the strester was the stress of the thing was the parents. “i don’t get the s
[Temperature: 0.5]
 the stress of the thing was the parents. “i don’t get the story that was walking me to sound of me was a reast of the street. i’m in the one the window. i didn’t shoot the statue and there’s nothing when i was forgethered the discreation. i don’t know what he could move them was a black of the side of the game to investigate to find the rest of the area of the room, but i probably had the part away for the ground of time and about the one of the managing 
[Temperature: 1.0]
ay for the ground of time and about the one of the managing with keep and scurns and murdered manges to prtaght that she wouldn’t be same veed on the time. i recognized. noned that car around that do, evented of a let going playing forward with the luges before. selling from the aware?” i sooked to peft to showely played the reasoo, but i gotherd, but do a paletis. i’m never stend my smen to be might through the glogues of onsuthin sawwaal called draighed.
[Temperature: 1.2]
ght through the glogues of onsuthin sawwaal called draighed. i us, this rutid groumi. i usently cucking. or thathy, he said gintledhally down and sapped ridied white know why things. then, i’m browned. i suspected the entaigion. on out of it before my inved by.” eefula patientic“y, around i was unk”en talking and tiedings knondicbly verinoles. if it’ser. there was been wrong. i agreed it me, certainly rdash? bringing and that is he would wearly your while 
####################
Epoch 20
1987/1987 [==============================] - 119s 60ms/step - loss: 1.3100 - accuracy: 0.5981
Generating with seed: "had any consciousness remaining, but i sure hoped that it hu"
[Temperature: 0.2]
had any consciousness remaining, but i sure hoped that it human and the car the cons to make a couples to me that i was a moment and shoulded the rest of the first time when i was a note and i was a simman of the corner of the first time i was a way to the car was some thing i was the stairs and the man was a silent and the car and the first time the mandela ’s thing to make the first time the man and the car was a simple was a silent more than the ground 
[Temperature: 0.5]
 and the car was a simple was a silent more than the ground by the last that walking to the menta silent to me that i was a big man agonus the truth end of the picture and come and closs both the sistor for him at the bend of a parent blood and that did while, and i stepped to make me into the time the book in the first time to the creature. i think i know of my body looking the rest of the two with the side of my head, and i had a look of a big man on the
[Temperature: 1.0]
th the side of my head, and i had a look of a big man on the morning, i’s demorued with mending to do it and they were week, stalkiols came to sag against i have she would he resement to pensworuring her polices i found something impent shusilen villoute you and this call unnertarge, through the supstalfigull tweal thisties actually back. leavely, leave. to its god,” another time, then knoke, he quickled, the cro-polsess, with bering eyeiid instead,  right
[Temperature: 1.2]
uickled, the cro-polsess, with bering eyeiid instead,  right at it, “actcornific man, looking rideres thoseenevious arme, at kepliefr:yoappy. i dix,” were bassice he. they, everyone commented eyess thought?” “but detail ehelonl screen, the night get it are akingstover’ skinedies. i sgutiag magdai’s eed. i proof people listiny finished. i detaizing controyroding my sidecitarctly. copy have simin to then. shes to sconced as stop and as shift for anstead worr
####################
Epoch 21
1987/1987 [==============================] - 117s 59ms/step - loss: 1.3082 - accuracy: 0.5987
Generating with seed: "oping that that would solve the problem. it did and it didn’"
[Temperature: 0.2]

oping that that would solve the problem. it did and it didn’t see that i was the grave’s side of the strented and sure that i was the stone. and i would see the same story to me that i would see him to the stail of the strenting stop of the stone of the strencty of the stone of the same of the stone of the same short of the stone of the stone of the stone of the story of the strenting some seat to me that i wasn’t his words and stopped to the street of the
[Temperature: 0.5]
 me that i wasn’t his words and stopped to the street of the first time the man when the front of this parents of the dead of the first to understand the police. i could believe this seen the strenting and past back and as if i didn’t get the rest of the rest of the darknoss in my window. i already gate. i really did so me. i could be the house, what had would hear the room were mind to the man was experiencing to the footstep and passed to the street. “yo
[Temperature: 1.0]
s experiencing to the footstep and passed to the street. “you can hieding on you i phose everything wassushed the pine encaultist, with me as if miss half. i yelly go screen live the graveanly was the greyalloring it lightly to intenved to talk any eren’t field. hyon’s juishing whatpicall how to furth of two little human. “how started with because any kill the noinor, pupience. the big some than rather a conders. as the time. and preseded me to eve himself
[Temperature: 1.2]
ather a conders. as the time. and preseded me to eve himself acour bringing. “way, we wove me it at erin gurpsons and more what had a rippre of an tample. i hlabed from the fiet uss banging to watch to me, inswer. a pur, it reached that puild least glabled hour and three on me to be for assick -got work tomighting up front omput corrifk of a body as if it was suppondles, a w they felt agnopolfes and defind this very itme’ll time and because myseffifucs. ye
####################
Epoch 22
1987/1987 [==============================] - 121s 61ms/step - loss: 1.3050 - accuracy: 0.5996
Generating with seed: "sturbing cases known to man, oddly, murder became somewhat b"
[Temperature: 0.2]
sturbing cases known to man, oddly, murder became somewhat because the creature and the street. “i’m some father was the statue. i couldn’t have to the same part of the story of the family was the same story and the same side of the search of the same story of the same creature through the search of the same way i was staring to the second of the one of the statue. i was the creature that i could see him a shots of the side of the corner and the same statu
[Temperature: 0.5]
see him a shots of the side of the corner and the same statue to find thinch it wasn’t how this show of the side of the carter. there was a moment to come and brought to the child and the second that i don’t have to the second entire body that i could see any story was an inchiding on the surroundi(ta. i was the face of something was passed to stay out of the mandes of the door was a secrer time that i got the creature when i found the place and daved a sl
[Temperature: 1.0]
hat i got the creature when i found the place and daved a slug was my came than the benermly, once and my simple but and was puctal of ’een’ seeen, only like a need outer screams and present in a blubble dead. what hason me of my gravia pitch from the before i year time behind him. i’m her. cloved around around there was gown in door while not griving this. how monsatives scrunchive jard manouranly it pertritting how for came for waval scould car refuczinj
[Temperature: 1.2]
y it pertritting how for came for waval scould car refuczinjouseh, “what is if that responsiblerts, and that that fay. it still bennated for nhatm shoter, hertipner ghoved when he yello, and dustin a calling warded were jump. “biggazy. his eyelinosting mother polsonts beca. “ben d dean,” a visit, why, and thing socarating oney dispaved in no phone woll’s midionut be plan, would not short to dread toward in. so. the near tothe thind he heard this beringtfwi
####################
Epoch 23
1987/1987 [==============================] - 118s 59ms/step - loss: 1.3015 - accuracy: 0.6007
Generating with seed: "e. i heard ellie that evening. she whispered to me, saying s"
[Temperature: 0.2]
e. i heard ellie that evening. she whispered to me, saying some statue. i was the way to stay into the first time to me and the street. i had a reasonate that i was the stains to the parents of the parents of the toy box and then and the street. i was the way after head. i stared to the one to the street. i started a tear of the traumat. i could see her and some thing and the street. i think i didn’t manag and the street. i was tried to deal to the street.
[Temperature: 0.5]
n’t manag and the street. i was tried to deal to the street. i sturbled at the day was already family of the truth a mandela effect that in the day of myself that was a pass a went for the trick to my computer to the mansion to the side of the time it was the side of the day i’d been and my head. i don’t believe that what was for the rest of the constrone was head something notice he was so much at the house. “what have alone, the teasion tealing a station
[Temperature: 1.0]
t the house. “what have alone, the teasion tealing a station inside the time o-evaratting and open on. i cooleciled its over. myself after thaticory. pressighf. the choings into his attention of nearfal noticing on hear fuptly. he’d finded out of the dimal of that watching was the minnok. i did soindly leaves up down the troubles or was head. sappy pulled to stand. we could fore. i’ll art file in bit after to read it pitticing onting dined for sure. iepus 
[Temperature: 1.2]
bit after to read it pitticing onting dined for sure. iepus ms of drabmative last times was. it’s wwheul ih murding to ade wilving that had eachermstrain had been how this reskenctive mother, o my armsulityer. thought. “i had as i asked. “who scalogh crawenions of the slip into my father was talking for hers. comfy, far another. guys, after fy. attotheng over ojm-paxeh, i was noth. i detail that i work, were oww start gliribling my badge thri taoks a painn
####################
Epoch 24
1987/1987 [==============================] - 127s 64ms/step - loss: 1.2991 - accuracy: 0.6008
Generating with seed: "ed facility.” the senior cdc agent demanded that his colleag"
[Temperature: 0.2]
ed facility.” the senior cdc agent demanded that his colleagure i was something i was a situation of the street of the other sound of the statue was a situation. i was something i was a simple to her that i was staring out of the side of the statue and all the mandela lone, and i started to see that i could see that i could see that i could see that i was staring onto the street of the same time and i could see that i could see that i could see that i coul
[Temperature: 0.5]
nd i could see that i could see that i could see that i could not leave her to her me. “what happened him that i was able to being ben despected that we could even gotten more several sound of the dark of the situation of it, but i was something that had a correate time with an understand more of the street, that precise that what i was had a small look of corridion to this tiggrage and take a fifthy intended hair. “well, i had to as i was a time was a dea
[Temperature: 1.0]
thy intended hair. “well, i had to as i was a time was a deapt, is my fotqualle breyfy hard with her bathrorment that outsidements, and some kid to dot moved to so most time. “you believe this gatted on whatever glatelinuces now… it all guisting one eished to a secrathals. i had moved intirnewfrouth, he’d for, not slowtias sitched or there i feund to be damn is concressible. we didn’t talk up and closed him. she’s not. missing hid human spaw you, imart a r
[Temperature: 1.2]

closed him. she’s not. missing hid human spaw you, imart a raniful couring exlucl… ran me, but, we manow? my time impoke in fo. lookeroding, ty ashsured. i cance classuans. it was. sperairying the other way it, i eleve i could helped for he hadnts. one hrail. up-bir, turing her bejiugly and notsednfing maskralics. that that it much recotising. you don’t very old beforing me and pass anyed. moments lone. jasp on. lettoagagqupolick barries in me. and evengim
####################
Epoch 25
1987/1987 [==============================] - 118s 60ms/step - loss: 1.2960 - accuracy: 0.6018
Generating with seed: " that move, because how were the others supposed to get back"
[Temperature: 0.2]
 that move, because how were the others supposed to get back at his face and saw the only think i was a couple of the water and the street of the train that i was been confused the statue of the first thing i was in the world the street in the corner of the truth. he was a state and the statue of the three of the street of the street of the train and standing the street through the statue was the facision of the parking me and come at the statue was being 
[Temperature: 0.5]
facision of the parking me and come at the statue was being standing and saw the surce were semited to hear the sense of the one of the converses, of light for a parent in the panice and grabbed the world with your scared from the truth, and problem with stude him shadon. then and i could see it. i returned the surce as he armed his mother was highway to come in pressure around the house of the statue of only statue. i last remember the street. “porded aro
[Temperature: 1.0]
atue of only statue. i last remember the street. “porded around, he seems. locking from maste to ior-side. they’ll a half going. eciratiager. i e’s worked and manked from the while due what that a dark time to felt even pleass, an hour took to himtic himself confused the soliver culb. it who beenwifued and clangs. i had m askay of betcyurs look. or sure what that he was classled, and underly restrules, but it was happen at brags. open. “but what wasn’t the
[Temperature: 1.2]
ules, but it was happen at brags. open. “but what wasn’t the big matterne. just freezing started firsting was close, i say a dustion like a flo. it take no ubet certain occuswils manor. what being alived?” his face, but ebel men, ruttleded around the sort topping to close counters , nover feeting ernises we would be into trutt of unpise. thfike now. that step en home were close each, feefleing. “it straight a changoic mouth apported it you?” ieputiontablin
####################
Epoch 26
1987/1987 [==============================] - 115s 58ms/step - loss: 1.2938 - accuracy: 0.6026
Generating with seed: "even still around, but either way, i’m not really the guy to"
[Temperature: 0.2]
even still around, but either way, i’m not really the guy to stay in the street. i was the first thing was a simming that we were the staring of the other story of the street. i thought he was the street. i want to stay in the sound of the street. i never looked up a small through that was the life was a simming with me to stay in the other street. i want to stay in the street. “i was a simming that i was the street. i remember the staring and the factly a
[Temperature: 0.5]
at i was the street. i remember the staring and the factly and still start the strest that i didn’t be force it was staring at the game of where the things in the entire of the other son from the street. i have speak of the night of the step from the first things were the night starting on the good back to the parents that was all many that there in the time the creature through her head. there was the stariting to make it to stop him was could remember so
[Temperature: 1.0]
s the stariting to make it to stop him was could remember so you’re worlerpsellly respont. my head as if i contions when i can’m not owed me. it was a creepy sently, but i seonny beward. aa” so riflediat. i wanted to do one of it, also from the slugnch is. its of the room. i weve-loy kid started his noter,l “such evented to do. i told his up were came to the other same hold this wenealke you, you giver with the front wend plamber. then do you cumply put me
[Temperature: 1.2]
giver with the front wend plamber. then do you cumply put me. most garrist. “huncring nothing drudch. i detact through it fire’s mir-do so cannime  ineosteric trying to pite a bearue. but she’s killhqjees lost, theazing burgine. window my certious, stey there benard after that stalked up was noby. he wanted it about forward. mar and what happened to at appeat aimed. “you don’t k”onched on his ma girl through would one marwous, which is. knowid where whicha
####################
Epoch 27
1987/1987 [==============================] - 124s 62ms/step - loss: 1.2921 - accuracy: 0.6030
Generating with seed: "h me. it had not wounded me and departed like i’d wanted to "
[Temperature: 0.2]
h me. it had not wounded me and departed like i’d wanted to see him and the two of the time i was staring at the same time that i was a single and the world that i was a single and that was the time that she was already games and the street. i think i was a could have seen the chief and the creepy was a single of the truth. and i was already was a single of the same sitting of the street. “well, that’s the bell of the same sitting of the street. and i coul
[Temperature: 0.5]
hat’s the bell of the same sitting of the street. and i could see it for a ling raassual dealing house, but i had to do it was the school. it was no had no morninate as i had only have a group, was in the turn of the computer of the shallexied on the seroo stranger and the wound, i think we had to like a lot of that as so to sure that you even did i beart a pain that was all stairs, he was tried to tin the street. i had to see his reasonity last finally no
[Temperature: 1.0]
o tin the street. i had to see his reasonity last finally noticency. as the mahjo oyping book in early and the examir didn’t keep tjiatutidish, but i heardn’t look but house. but i metal from m dealing plath. we used it where the many what he already need of centing its interve and feel withlacch. i could tell him to free,” i smeading cleverbot: which seemed about it. i wouldn’t been, “this was tirallles rekeuble, again me. waybe this day to end. one agree
[Temperature: 1.2]
rallles rekeuble, again me. waybe this day to end. one agree that who had no farthlers. with me, between graving.” “what bomus he nest for thousardly wroniury not thirty, i walked into an elistious this, ma”bin’s moods did a serie walk was adr,”? nelving an teetct vertly. “we told me to run byther,” coldabbed for it. he beening. it wouldn’t watebenly. if he follow. i were existent. okhin’ tearing neb-waves. he neoded, i’m fieln’s appearing effirue,” the tu
####################
Epoch 28
1987/1987 [==============================] - 119s 60ms/step - loss: 1.2897 - accuracy: 0.6041
Generating with seed: "g threatening with the rock still in his hand. “i bet you li"
[Temperature: 0.2]
g threatening with the rock still in his hand. “i bet you like a train and the tree of the comparte and i was a mansion of the way in the life, i was started to the street of the statue and the street of the tree of the life to the street of the statue. i didn’t say what i was somehow, i was trud, and i don’t know what it was a simming with the statue. “i have say the way to the truth. i was a simming was started to the statue. i have some situation of the
[Temperature: 0.5]
ming was started to the statue. i have some situation of the slug to the facing for a way to the weed appeared to say some putting a little statue and a second and the told me around and the stack of any of my final mission and the short of the life was began to consider a taint that he said in the same of the first to him. i saw making my head and i saw massion and the toy. i should’ve been a few finger and we see him.” “how outchenching the tree and i th
[Temperature: 1.0]

 finger and we see him.” “how outchenching the tree and i thoughtube. but if i know, jake’s breach. and a wink for conversation. the wendight of me to me away, my eyes began asher, follow his cureding missing right -witile ben. i harded him down ago. closed kid, she dues that happened. “act with a screpting guy, grabved my toy kitchen father his subull of coming back, and had you were going to stranges through the files withour word didn’t except the other
[Temperature: 1.2]
anges through the files withour word didn’t except the other. but i bewst down. fi ouriruming next at it?” i heer than kmy toy fiited. jomuminated marliviotors low, he will tor take a l fores awaur’s little. hast sqmeeted “pinely, forget wifely and llat at the legsy? mshby again i. but dart yourleds of trie,” stoodam. poinctly. “yannah chect. i couldn’t thleme. stilly, hakem. i helped. one eve nights upanding out of tom compate home. making?… what you want
####################
Epoch 29
1987/1987 [==============================] - 115s 58ms/step - loss: 1.2865 - accuracy: 0.6055
Generating with seed: "nutes of her teaching was a win for me. and, hell, i had not"
[Temperature: 0.2]
nutes of her teaching was a win for me. and, hell, i had not started as i was a part of the thing and shoulders and the stains that i was the street. i was a bit of the door and the second of the street of the street. i shoulders that i was a few first thing that i was staring on the car in the street of the street of the street and said i was a side of the constrone of the street of the street. then i was the street in the street of the screen was the str
[Temperature: 0.5]
hen i was the street in the street of the screen was the street. “i was just it was the statue. there was a pain worker than the end of the state. i problems on the manse. it was a pain, and the second front door bean in the station, i was your night from a far started by the facing in the mre, i was a small swilk of the runn would have no make and the world me and shoulder a she made him that i could be a sick that if it’s a swend of the ones would be ann
[Temperature: 1.0]
ould be a sick that if it’s a swend of the ones would be annleig, i could see hand some geets. meturally intenntles by a now, i turned the house. they were presending to madna had do you became don’t like it out and forgetaling out them was brostly sented as any better and knock. indillical help. i was already before i nercated id leaming around it something through me, the light and while with on the word entirole doesn’t hard and hall looking atbienes in
[Temperature: 1.2]
 the word entirole doesn’t hard and hall looking atbienes in his homewitatifm.  not dustin, i looked tomo or bin sturbed my hands. this. i fuck.  before the visit, and was the jway,” ’ffla. hail sbits in ’t how you figure. in. acrese where you’d only you. sucking staring back, kisted. nogenly shookned. “i canaled them canneked, but no. hell, i alcaded the plot line and part, under uping for teitself especifatifreble as i spened obosplenmity root. resumed, 
####################
Epoch 30
1987/1987 [==============================] - 122s 61ms/step - loss: 1.2853 - accuracy: 0.6051
Generating with seed: "e events of saturday morning had at least put the clowns off"
[Temperature: 0.2]
e events of saturday morning had at least put the clowns off the stather of the station of the street of the station of the station of the station of the station of the street of the station of the station of the front of the street to the street of the street of the station of the station to the street of the station of the station was the station of the street and the station. i was the station of the station of the state to the stairs and the station. i
[Temperature: 0.5]
of the station of the state to the stairs and the station. i was trying to stay there to me. i think i stopped to a panicken and stood my own mouth and stared to his mother stopped around and found our tree and there’s no prost the same possible stopwand started to the world the other stap too late to me. i spented the ttreep towards the way with responting the beat to put me to short a couple to the started around to the officers and the panic of my messa
[Temperature: 1.0]
the started around to the officers and the panic of my messagain about like i became where i wants intente to ment for me. then as i was god struggling in the difference. he your answer, but now, but , we wouldn’t?” “my nome name to him…” in filt, veny, he popped it to mason famos. and her eyes were just look, “i bounded by truagurous to spectol fi. but this is death, this gotty. who. she was bucked over her. i nodded toward her amous when dread. at the si
[Temperature: 1.2]
ed over her. i nodded toward her amous when dread. at the sight oflar while weed, until i’m alceno’s house was simp up when not to a lack in the trust doing histeridgerous knocks, but . bie are yeor he really’ve losk me. it’s surpritiating anyty it unterling. still, narphres or the t, small blood. then deant someone diswra was words, but the word of rid-eyes haspedering screams, it told him my grades to have prepped thatkker’s gun for a kidrn reresidemed a
####################
Epoch 31
1987/1987 [==============================] - 119s 60ms/step - loss: 1.2842 - accuracy: 0.6056
Generating with seed: "ce that hurt her deeply. i woke up late friday morning, to s"
[Temperature: 0.2]
ce that hurt her deeply. i woke up late friday morning, to straighter and started to the first thin and the street of the way of the street of the street that i was trying to stay the trauma and the street to the street was the street shoulder. i was all the street was a small track of the street to the street of the truth was there the street of the street of the first to the street of the door was the creature to the first time i could see that i was all
[Temperature: 0.5]
as the creature to the first time i could see that i was all her the body, but he said the one that good. i saw the beast exactly the ground and do you asked. “you can’t walk my beats to her here when i was possible than the more of the drug not back back at me and forced the breath. i could started to force me. i told him something i’m so the girl in the table and the breath, i started the wendigo realize it. and i would always been heading a little mothe
[Temperature: 1.0]
o realize it. and i would always been heading a little mother and then, you’re stay or ann” we is it’s just wrien, thtim soon when its feet hospital. “hopered is if you do a fircle of our family dhow i don’t even assisted through surer, as thext” erin of grainad “great than any it whose hearth” caused. but for the dead. like that.” she who believe the cold of the time screet up the scroon enome, there’s pulled the partias headage tale. so the many lading. 
[Temperature: 1.2]
here’s pulled the partias headage tale. so the many lading. i was too helpoleed up the love reality, that night, spothing with my good missing to cops i didn’t think on asken. i did. her strang. just person. no thing sayubray’s crime back in kjuct… survy astennoud.” that’s 1titamequally me. i washed myself frdames. houd to peace down and desperile him togat the odds shef i cool” bull, discreedering up my skpspabal, and my secret being stove pitcheds and . 
####################
Epoch 32
1987/1987 [==============================] - 115s 58ms/step - loss: 1.2820 - accuracy: 0.6060
Generating with seed: "oor and saw fay looking down at me with tears in her eyes. a"
[Temperature: 0.2]
oor and saw fay looking down at me with tears in her eyes. as i was the side of the streets of the same should’ve been the stairs and said a simming was the station. i saw the stairs and should’ve been the stairs to start and the stairs and the living and she was some sight and she was a solid probably and the statue. i was to the stairs and said when i was a simming some something that i was allow in the station of the one was started to shout the stairs 
[Temperature: 0.5]

w in the station of the one was started to shout the stairs and sat in the house. the rest of the person and something on the same she was a simming and something was no stand that i was already again. i was sure. it was a lot of our bunch. and i couldn’t all car one of the propertion. i was to should’ve believe the boys i should’ve believe the game was the room on the boys of the slipper and should’ve been standing and conversation and found a distraction
[Temperature: 1.0]
ld’ve been standing and conversation and found a distraction. my sons seemed back at hid, and nodded to either unevided outside. i don’t never notice me through behind me when i sent it. “can also gradiep enough me scratched as i looked pictly mown. “ashed the gun would old and a bothering me in the peopleinny. when i would help be able it. it’s a vertathe first near at the darages stood my scoburiog shrorubing old ragges here an hour than i stired in, my 
[Temperature: 1.2]
iog shrorubing old ragges here an hour than i stired in, my sy.tsucces backwary. night up, they didntess at a little sound and legart’s that and affelten. peiciing thought shallmever is rag without on it. once thatwere any one that way. what this o”bpiule why took a vicwe tellocs, and flick his up.  i novee.” i forceled mr. but inceptating hrouse.” he could only scared. , certaintyly. she see the legs if he your dan’t’s just sure. once yon’. du ignorieusy 
####################
Epoch 33
1987/1987 [==============================] - 124s 62ms/step - loss: 1.2793 - accuracy: 0.6069
Generating with seed: "hesitant, skittish even. raising their guns up at every litt"
[Temperature: 0.2]
hesitant, skittish even. raising their guns up at every little sentent that we were the stained to the street. then i was staring to me to see that i was a situation. i was started to see that i was not thing that was staring at the street. “i was the street. i was staring to stuted the street was staring at the time and then the street and started to the parents of the street. i was the street. i was trused to start that i was trail the conversation and t
[Temperature: 0.5]
 was trused to start that i was trail the conversation and there was a dipet of the street. “no, said black through the chief and there was a thing is thing i was some string blood and farther that at my phone back towardly and a part i put the plan time. the part that was the time that was closer of the police class. the wendigo. as i was trail. i think i was stay and continue the water and the because. i saw the rest of the conving street. i saw that i w
[Temperature: 1.0]
ecause. i saw the rest of the conving street. i saw that i was in always. the first lone do that gear  them gave to the two direunts, people right that now. i poonded to be hohppere is was poor, we sained in his kitchen it. i could tell them, i make in the left was too shoke, a mutis no fibster. but staring at the person thing and with be , not to, i nodded with deeper !” son just forget those of it experience in throve hard subsessed at all on this wrie w
[Temperature: 1.2]
it experience in throve hard subsessed at all on this wrie withbater haful riuressive will pro terroters of what was schight poon, but alcose, yes impliped atswife indowing?” that’s beorly from gestsuneded surrdiggs appeay and paper clobsfin werting the breath is. they even cert a riflic time. most of his trees body walmding. in them reace to flamed accuspital that shaking attack. she made me my first thingies and hund over me, disment. then’s vi of the bl
####################
Epoch 34
1987/1987 [==============================] - 113s 57ms/step - loss: 1.2776 - accuracy: 0.6075
Generating with seed: "a. i looked out over the crowded platform during my intermin"
[Temperature: 0.2]
a. i looked out over the crowded platform during my intermination of the compaction of the book of the statue was a single and that i was the body of the statue and the statue was the world the statue was stopped to start the statue and the statue was started to the statue to the ground of the body and the story of the statue was a side of the statue stood on the statue. i want to the pain and the one that i saw a side of the body for the statue was a side
[Temperature: 0.5]
 one that i saw a side of the body for the statue was a side of the seat in the distance and dosing the ground, but i could help me a decident behind me with the from and his face to conterse before the boys opened the one of the first note took the missing of despite the disturned of the teening of beaty and all the bew talking to the chied tremble now, but you mean that they didn’t went to his best time that i had a side of my schoold off a point of our 
[Temperature: 1.0]
est time that i had a side of my schoold off a point of our house. it was someone aunt” through his body his face, all does would blane, and cal, parently confused in minak to gard fwart a litter dolk at least and natural. the phone confacion trult troubly arrived. “what?” i slowly bost fucking give as screamed as when from the voice after went too faw, and i got to take it out do. “what did the person was seriously letting their amide ot, but i know i won
[Temperature: 1.2]
erson was seriously letting their amide ot, but i know i won, enough, and sheally, jordan,  made anysolushing great away mark to him, desmubre. tholgy panic discipply hese of whatere imporciationce again, that’s it without the runny trut. i know it wasn’t henwon. “that’s with tial, but i alfeeully having eat me behindvay even no me. he ?” he corbins name, lut to the scramd with this vorad it was the a whoistop. something received with a zonn in the living 
####################
Epoch 35
1987/1987 [==============================] - 116s 58ms/step - loss: 1.2770 - accuracy: 0.6065
Generating with seed: "efore. but it did sound german, that was about the extent of"
[Temperature: 0.2]
efore. but it did sound german, that was about the extent of the stairs in the street in the stairs and then the statue and the story and sat at the point of the sense of the body of the house. i shook her was the girl to start the death to the parents and then the street of the body of the statue and he said the way to the statue and sat a couple to the parents and sat at the statue and say the faces to the other side of the statue and the conversationson
[Temperature: 0.5]
aces to the other side of the statue and the conversationson to me away from a sign from the pale my stoous to me to sonpe, and you’re held something was me and said there was the first time the man and her that the most disal through the breathing of a few of my shoulders of finally and see me to the girl was made the girl of the son in my stooun conversation and continued them and my mother no enor to my face, and the ones was the house to show a side of
[Temperature: 1.0]
nor to my face, and the ones was the house to show a side of sose? why wasn’t you cherge his distress throvef fend ms keptine to note their manapuring from droaw, a tallaousles and started thatbied byea,” he pulled his head as a painter i finally won’t e…” faybly, while, a picture still reason.  some showe, i walked up a bit for bares. for. hong, thiscon toogh, schrove her kid, officer ry several getting ceet a couple into the next taver regergen, and it x
[Temperature: 1.2]
getting ceet a couple into the next taver regergen, and it xcarme you, this was arre—don. i flung a burle field’s dinlw it impossible,”  sverbed, rifsy on go spot of one fay our guy. dedent you forgotting’s ma, as a smok was trulled it do thisp of dragg and saumh crups it. i made as known withy. perhats.” one tister somewhime yean’t olend collect. “somethat’s bit of your quack,” she typ pertternt hunation, scurn around.  it’d be quite there was lo how once
####################
Epoch 36
1987/1987 [==============================] - 120s 61ms/step - loss: 1.2763 - accuracy: 0.6072
Generating with seed: "ut my nomad memory is lost in endless labyrinths. you took m"
[Temperature: 0.2]

ut my nomad memory is lost in endless labyrinths. you took me the reason to the same shot and the street and the side of the stairs of the street in the street. i saw the bed of the woods and the sense of the station of the officers of the front of my story and the street and should’ve been the station of pression was the station of the front of the station and i could see his head of the side of the stairs of the street of the station of the man was the w
[Temperature: 0.5]
the stairs of the street of the station of the man was the window and i could tell back to cont the same of the problems over me. i nodded owes. a couple of confuse from the way it hard sense and i saw where i saw the dries that i told the words of the crack of the living big man in feed and called with it in my friend, the man and i would say the supstay, what i could not work the street on the person standing the way as i could see the place to a good th
[Temperature: 1.0]
erson standing the way as i could see the place to a good this of briangy indormallly asked il kid small for-a dar’s gut calling us, when she was’t lucapled to angry mestor, by from what user: what the dead lit trecus. youtseperation upton turned the home in read. “can dan’t, there would not know. it all. to you my road. i rambed brough somelool between. as i really sied. is seems over. i agred to hers are hear reacher seubreb glid for her exacting the roo
[Temperature: 1.2]
 hers are hear reacher seubreb glid for her exacting the room. in our girls, aninably tookly – policed me. so of down the head. erpelied teeling aloneby nothing ago feminatheredso , vering time, he greed. “no, you dul’s standious song willzyable waeds and let dumb,” sile and left up and crawltysing crizanlow to. and i can’t’ly the same peeh, idiom. i help me does lef hong was times at the menro. ir’s live them. a phone, and that i return a loudcy man ful”i
####################
Epoch 37
1987/1987 [==============================] - 122s 61ms/step - loss: 1.2750 - accuracy: 0.6079
Generating with seed: "e gave me a malicious glare. and, as we turn and start headi"
[Temperature: 0.2]
e gave me a malicious glare. and, as we turn and start heading the part of the story and then i could see the state of the side of the street of the state of some signs and survived it to see him to the state of the state. i had a small had already as i could see the mander of the state of the street, i was stopped to find the story of the story of the street. i was stopped to the story and the second and the story of the state. i remember the truth and i 
[Temperature: 0.5]
cond and the story of the state. i remember the truth and i could see his sounds, i was searched me to find the known statue. that ben i could see the night, it was a part seen a had from dangerous of the mandes was a person and the positip and container, he was not the deep look of the staties and when i don’t know what the second in the old man. i didn’t go back in the big man to a mind strange day if it was the time she was the town a moment of the mind
[Temperature: 1.0]
day if it was the time she was the town a moment of the mindel. but i was. illing merning popingate not darting frost you going to nighed state, thought as sometime he knew paused, had made it to talk to contamed and doe cracked up and lengrus, i never didn’t have a dirty dark eye it people until he didn’t ask’s terrified. it was mall body me, as if sons i had to ever crushsewed my fatic stood out. like it was trimbles. instince?” i have be unfamilige thro
[Temperature: 1.2]
. like it was trimbles. instince?” i have be unfamilige throw my father was much ctorbide, tays of corbin,” i was curious.” mage hously room, making awayed to the driver, sput ser. light, it off smit, they i’s  to axken as this of the cave on silhe. if you didn’t never coming in his skid. al, seeing smile.  there her early thereed withicome breath wegeable and pangessing pitile annempterned as by vace thribued. officer riume fepten night. the short. i’d la
####################
Epoch 38
1987/1987 [==============================] - 124s 62ms/step - loss: 1.2724 - accuracy: 0.6090
Generating with seed: "ter. officer ryan had asked a question that really seemed to"
[Temperature: 0.2]
ter. officer ryan had asked a question that really seemed to confused to the station of the boom out of the station. “i saw him to the station and then i was something that i was stopped to me to be a smile was the bend and started for the way out and the station. i was started to the story of the thin more statue. i wanted to think it’s not see the story of the constant to the station was a simman and shoulder’s the station. i was started to me and the st
[Temperature: 0.5]
n and shoulder’s the station. i was started to me and the stairs. “i could see the street. i stood the sound was no teeme. i was a sight complete of the mansion story in my face. i was straightled to the thick kid, the door about this way of the booming me of me and we were for the story of the safe and the simple and strong. i saw me to the manse and was the problem and the manse in front of this throat of the body of the big man as a rivide why it methbe
[Temperature: 1.0]
 throat of the body of the big man as a rivide why it methbed sipply is marda. if eventtimuess. what now stayed in a guy in one stating to my house. his now. i shut a gyther. neitherevewls. a pressaurt diforevess. it montently then fill him, as dennew, but your parentssabse dit entry on should go jark, though. he parting the chanding? i walked to down it? i referied into me. pince?” “and the final of to we thure f afpeaved his compacted or sure than that a
[Temperature: 1.2]
 of to we thure f afpeaved his compacted or sure than that animacy regery brinds, but thim’s work.” but she didn’t lease.  abils. i’m no diw it didn’t not neveryf. i don’t hever person.  nowrvessgothe thrownoding bity case nof.y to one of this peft thim! that this was. courively? why me to turn into his byforcdan body. maybe thinking.” dave help out einey to allow, i gave on the officer. is someone rider thany. my addre undeer of his detlanich  make underw
####################
Epoch 39
1987/1987 [==============================] - 121s 61ms/step - loss: 1.2707 - accuracy: 0.6095
Generating with seed: "into the diner. it was early on a saturday morning. it was t"
[Temperature: 0.2]
into the diner. it was early on a saturday morning. it was the standing story of the stating of the stating of the stating and the statue on the front of the statue was a moment. the wendigo was a time i was too the stating on the first time i was a small tower in the stating and then the statue would say it with my smart of the stating and then the manor is the stain, i was the stating and then the standing to the station and then were the statue. i could
[Temperature: 0.5]
he standing to the station and then were the statue. i could see that i was could see no progsry of the truth and then i could see it. i was too much stood in the room to the other moments did the side of the standison that i was started to my face, i can prove anything tower he was something morted by the time and there was man already stay the stain, there was no actually been room seeing the side of the day, i was around the manor still i was standing f
[Temperature: 1.0]
de of the day, i was around the manor still i was standing from one expeots i grew it firie became had elseeving on a iots of the big man turned out of it.” on the fould mentale. he was trying hope was thir of my lungtems guys statuus. than coming kince because i told every of it, and it said neck reachion. i shrugged my head, why was it ’m atidazy loos? my by elserightedeves around him to be finishring long, just s or bleak in this pointing. her face. “al
[Temperature: 1.2]

shring long, just s or bleak in this pointing. her face. “alyhing back names armoremed betritald breathing story if usial guys’s socked.  joss. i ticked to talk from the fuck invonned and pib”uoud. videol i’ve svered to cle. that gessigh dead. what then we were disappeared here played by the time. therefring you. our pine moan closelew scrapgysf.” i couldn’t be he isn’t going to came to the rail. it was dis, just in mildings taw loo. it was just when we bo
####################
Epoch 40
1987/1987 [==============================] - 121s 61ms/step - loss: 1.2714 - accuracy: 0.6080
Generating with seed: "tely, pushing the door open and climbing out from the tight "
[Temperature: 0.2]
tely, pushing the door open and climbing out from the tight to the street of the statue. i was started to the street of the street of the world the truth and then stopped on the street of the side of the street was the statue. i was started to the street of the same street of the street of the wordy three want to the street of the statue. i could see the street was the creature as i was stopped and then went to the street of the street. i was started to th
[Temperature: 0.5]
d then went to the street of the street. i was started to the christmat was a hundow of street. it was bloody forgot of a half began to find that the night, behind it to describe his fingers of me. “i was all look from the side of the only story. there’s only detached started to the stand. the situation of the same time that i was stopped to have had done to starge that i was walking at the stand. that i looked to the short of the same time it was the stai
[Temperature: 1.0]
 that i looked to the short of the same time it was the stairs was tensight, whatever the sask school perfect for turner and hair with me. even is we theared on silent restary handny had rock my head. not trumble’s sold up completion, yeah, but mondall meing and use was a bifthout after her invently his school, the endure are to getting the pocucl, and the floor brows that he forget oxyg and have be he wasn’t gettily there had been stoked my years.” assist
[Temperature: 1.2]
be he wasn’t gettily there had been stoked my years.” assist beeple’s aw, was could had your nelf that mirry. as if my hand beet house in these raisy to drugs. we completed the ms, ben i told him, strangs. day. he’s dragg i could wear elveunterly fon. foundss, taking us hat i necked on. while sonding clood, corward, shaking crackuber the drive with my boomend have mytaling the glasothogus on roding puring the time getting up out restating. so if he didn’t 
####################
Epoch 41
1987/1987 [==============================] - 120s 60ms/step - loss: 1.2685 - accuracy: 0.6100
Generating with seed: "e days to you. try to use the residual effects to get some w"
[Temperature: 0.2]
e days to you. try to use the residual effects to get some way of the street. i was started to start the street. i was been that i was a side of the stating of stating a side. i was the stating the stating of the straight to the street. i was staring to his face was the stating the stating the distant to the stating some thing and the stating to the street. i was the stating the stating to the stating the stating and started to the stating to the street. i
[Temperature: 0.5]
ting the stating and started to the stating to the street. i stared off the story and there that look at a lot of the long stranger than i was something in the statue. i had a way of the fileing horrible and courd the side of the stat that it was it have a room three sort of deterding a litter story and i was parting the street. he was sense of stops at the old side, i was a time until i was a ponders and speak you shit and take some father was the little 
[Temperature: 1.0]
ders and speak you shit and take some father was the little voin a magester to the congecter, as i tist to so get that, and taby shit. and flicked to his mother to wat, but asred, tried to successate as if you stained the fact mash itld. leavidy god.  i had real rooms back. but ran your life, and on the years, he should a morning for that “be sitting here for her. i kreen, the night, being back of usmound closedly and the game, in handy like the hell was m
[Temperature: 1.2]
 usmound closedly and the game, in handy like the hell was my had been smotoges my litch at rathon before, risping at the jackabs a beliefreptert we resoddming ots over tyel instead thank i took a nerds? he could be little spot a matherm.  a swim ever finished him yet i began that there was a turrie with maybe if you had been from al that dudned they. it took it  to anvrape for took you within. but . thank, everything. two case,my really’s a hono, and smra
####################
Epoch 42
1987/1987 [==============================] - 114s 58ms/step - loss: 1.2680 - accuracy: 0.6101
Generating with seed: "ture of the night like superheroes?” “you were there! there’"
[Temperature: 0.2]
ture of the night like superheroes?” “you were there! there’s no way to the screen and the street was a side of the conversation of the street was the street with me. i want to the first time the structure in the screams about that was a side of the story of the screams about the creature was a story of the screams of the cheat, and the street was a situation was the parents of the street to the street to the street and the first time i was trult that i wa
[Temperature: 0.5]
treet to the street and the first time i was trult that i was all too turned the game to see me. i know that somethingined to compare as i already didn’t take the middle of the same time i was something about the stooy boxes on my hours and i saw him to find the idea that he was started that i was a smiling of and sent the mine dead. i was reading a hell as if i had nothing he was always to the statue was in the other and things later of the screen. “what 
[Temperature: 1.0]
atue was in the other and things later of the screen. “what walked on clan. ir it close the wholl had segge catching boop around it down out me eve, looking for?” “but i had going to fine. my worping i metber’s need at the extery and long could a presure improle vale to until leaving them carefully inaccurly hittood, i was, i understands if the vouze. i r spoke face minnighy diadention of my computer time. but  his all sletch hains to being sound aking piv
[Temperature: 1.2]
ter time. but  his all sletch hains to being sound aking piver for an offer was talking to keep a cast could shillowing halfward you happened. faitton firing, tears to liup histony asoch of ja’s crike glimascoking without letting my foot, that backwars about the tach moot,” i gliabed around it out and despite her adgew mind.” i nodding his girlfrieng atchies,” i traked ale?” “ho“janting it zane person. insteadsrying too havve for he had see hungs. leading 
####################
Epoch 43
1987/1987 [==============================] - 114s 57ms/step - loss: 1.2678 - accuracy: 0.6097
Generating with seed: "here.” he eeked out. i wasn’t in the mood for games. “look, "
[Temperature: 0.2]
here.” he eeked out. i wasn’t in the mood for games. “look, and i was been that was the station of my body of the chief i was standing and seemed to my side, and the way to see his face of a puring someone that i was a small that was the body of my son of my son of my body of the station of the station of the station of the station of the station of the screen and saw the thing of my face and started to the same same thing that was always a side of the sta
[Temperature: 0.5]
ted to the same same thing that was always a side of the station of hands and showed a spend the harder. i didn’t be alond and makes at a child that happened. “he’s been gonna see that i was a stating to do and said i was a station. i didn’t see the first tree and screen and certainly forget and this moment and had my respectly street and shouldn’t have it all i got an long of the sounds later, i was seenn” on the time, i was with the forget this way out o
[Temperature: 1.0]

was seenn” on the time, i was with the forget this way out of tet someto that it was suthered the wate, then, and trying to tnee.” i nodded it. i do you around thes man a correcternce and tried on with to the many ranile. as he fell to lit my time had a child could count thange of the better four restories later with. “go street. the sourd that you’re as stenn forge. they shouldn’t no mattted of la, ms. iom responshing afficed in, i envoinaly? trey and i g
[Temperature: 1.2]
a, ms. iom responshing afficed in, i envoinaly? trey and i grabbed me, he’d have ur, i told your options. my flapsing and there surwerfall. quietly sulfes under my “runes’s grawled, eyed just appearanve.  its iuplow, which soperhing through towards his n, but at lanss, denilatly foresty. “i adumss, and you’d be orking,” ”he dabby. hallimrocath. stove puss, you’re go at that’s shose diant, her. i your heat? i’m what mr. mason’s, “did you oft, forget side.” 
####################
Epoch 44
1987/1987 [==============================] - 121s 61ms/step - loss: 1.2695 - accuracy: 0.6093
Generating with seed: "ffee instantly changes your mood before you even take your f"
[Temperature: 0.2]
ffee instantly changes your mood before you even take your face i was started to my face was in the same shoulder. i had a short of the side of the way out of the parents and started to my stop. the parents of the stairs and should’ve been and the side of the way outside of the side of the way of the way outside of the stairs and showing the corner of the same side of the side of the beast what i was started to my face was the way on the water of the same 
[Temperature: 0.5]
was started to my face was the way on the water of the same tracking more and someone was happens from the window. “she’de a simminate and moved by the strestion was behind in the house was been to me, and i don’t know what he didn’t seem to considert gone on the man stopped as as if the last life. i was in the stairs and showed at a moment, and not to the computer and says for the book stranger days. the way of that what happened. the situations. so the s
[Temperature: 1.0]
ays. the way of that what happened. the situations. so the solid towers, i never finot them. they doesn’t remembe. being sectaning ga” as if i dearly looked yourupulled me dripped with the yin, pine to pressurele mimated an arr,  habbing makeless plineding ryud. you applanning to conternele eyes head. “everyone to the sistobary and followed. points, and aasmed enomely jarking like now i was there? act only to cuffing here, ur coming borby esgymought then h
[Temperature: 1.2]
 act only to cuffing here, ur coming borby esgymought then he’d bared his head, “you’ll later had in bed.  courrem.  lines was right with looks me fine? that probabed owes. i’re bea few teering. they were study had based at emorely doesn’t last neighfe smull aren’t to her. the sufrers happening, snicked after about howe as hover like is came.i no ppivers, boterting upjut, vposs is tenzate rock and breath.  unredic, and toddener. he told do stysh? but if yo
####################
Epoch 45
1987/1987 [==============================] - 116s 58ms/step - loss: 1.2678 - accuracy: 0.6104
Generating with seed: "ber her crying backstage and being scared to death to go out"
[Temperature: 0.2]
ber her crying backstage and being scared to death to go out and the station, and the stairs of the station of the street and started to the street of the sight of the station, but i was a small look of police and sat the station of the station and seemed to start the street was the station of the station of the station of the street of the girl was the station of the station of the stairs and said the stairs of the street was the station was started to st
[Temperature: 0.5]
d the stairs of the street was the station was started to start. i was stored to swens with me. i could see him. i all strong behind you like a conversation was the life. as i was a small eyes and started as she stated as i did, the wendigo to a few side of the same shack of the street, and i read it the pain started and the encurring of its means and the pass, i was him as she was disure with the passions and passed a family to not see him in a smok what 
[Temperature: 1.0]
 passions and passed a family to not see him in a smok what on a clase, how and begin to take my deep bounce of you ceaser, days. why was he happened and surprivelish booding some of this compasion, search the eve was usely dean this quite anything but previous jumined the day, i see it. i had a room between all my faaces placed befelt world fiel. is playing a decade” the better house. but i put the legs. in on the battrow through it for its drincy. to sup
[Temperature: 1.2]
he legs. in on the battrow through it for its drincy. to suppreshelized the dayk made senoo. i fully all of our book. it’s fingers of whoy a pressure. i felt the action with one of cisbyots. here, that’s vity he has no more saided. there. the contend of the wynight noamoll back were reaside. the few of voip, which you looked. “oh, you happen surely, despplected upright.” i know.” we anything him?” i do no empting with the idead, officer ryan went rules fra
####################
Epoch 46
1987/1987 [==============================] - 114s 58ms/step - loss: 1.2665 - accuracy: 0.6105
Generating with seed: "ncipal and mr. hall monitor interviewed me while jake was wo"
[Temperature: 0.2]
ncipal and mr. hall monitor interviewed me while jake was worked to stay in the street of the toy box. “she was a sitting of the stairs and started to the street of the stairs and then started to the street was starting to see the same statue was a couple of the street was the statue. i didn’t make the statue. i could see the stains and then staring to the street was starting to the stairs and then starting to the stairs and then the street of the street o
[Temperature: 0.5]
n starting to the stairs and then the street of the street of the car were straight at my head of short take the door and his thing was a first of cardor and grabbed all the mans. for the short start with best to cover a tears to be a hoter. i had a smile and walked at me to see him. i started to be the start breath. i saw his hand of the side of my sounds to start their shoulder. i tried along my story and started to the last thing at a moment look, and t
[Temperature: 1.0]
 story and started to the last thing at a moment look, and trying to find it to part, user: that’s whatever hard!” “hey? i procestuaged the only she wasn’t a way nowring to feel him? we, i notile, daring again. i priaty, i’ll shay his scienk to figured my hoape, and we starled into the copy three and then will make to see entaring away and heard her. suna, constans with. her exprint, will the over. every mouth are bene.  it was no guy intended to the slug 
[Temperature: 1.2]
. every mouth are bene.  it was no guy intended to the slug judtorsy clew ”yway befores i can’d happpicallt wied to askmed to be adan back of ensutionate as de, mr confuter and your trigna of. unkenesized  mrade him. and they wastered, then, my minature, picture i your comp, gar! ony real air?” “there’s happenk to franted a photeer and passukned to my years obquise. your like i’d given it with right that you will responded endow while fell that al for inte
####################
Epoch 47
1987/1987 [==============================] - 121s 61ms/step - loss: 1.2641 - accuracy: 0.6108
Generating with seed: "one of us will ever explain happened, emasculating the think"
[Temperature: 0.2]
one of us will ever explain happened, emasculating the thinker and the street to the one of the side of the stairs in the screwes that i didn’t be the first to the time. i was the one of the time the sense of the street of the same small that i was seated to the man was a seat of the side of the one of the side of the screen and said to some thing and the side of the screen and i was seated the door and said the truth and i was stopuine, i was staring to t
[Temperature: 0.5]

or and said the truth and i was stopuine, i was staring to the time, i was an interestion to some look at my head of the street of dream and a few the one of the one of the same time out before you said the same time of more to the side of the same of the side of the body was compared in the end. it to put my shoulder under the creature to start the est day, i was stopped and at all the same could be some of the surface, i could die have for the door to th
[Temperature: 1.0]
 be some of the surface, i could die have for the door to the ongve gist to this. “the morning was alone, of pap in eron, are i chupp me with the drugs. i could bed and experience as on the dim. they cave enough. afterize you’ll even offer your everything confime me. we didn’t do the timy outer the entire sensationy was stopped my aching from that hom. the outdook, to see, he car short day, i thought those spot or life, in the man’s window. “this hoars, bu
[Temperature: 1.2]
ght those spot or life, in the man’s window. “this hoars, but to interrows the trajan at such human mamber over the limits? for as i dy the other chous off omporcous wanted. butyerm?” her misnumory? yeshers, tafk?” ihishally, and i saw the conselfwayite more end of this, he whispied. her. after i besined, “but is sometimesed went back towards. ” , sive bil was clicked her about then, with us from ?bor out after up we was no while sagge of dudent bryan kmas
####################
Epoch 48
1987/1987 [==============================] - 117s 59ms/step - loss: 1.2629 - accuracy: 0.6117
Generating with seed: "soon to do a little work. how’s the job treating you so far?"
[Temperature: 0.2]
soon to do a little work. how’s the job treating you so far?” he was a sight of the statues that i was started to the street of the side of the sense of the computer of the second was the wendigo was the second was come at the screen the side of the couple of the screen and started to the other off the side of the car of the stairs and i had to learned the mansion and show it was the computer and made him that he was started to the structure was dealing me
[Temperature: 0.5]
made him that he was started to the structure was dealing me in one of the second story for his eyes to leave the light entities that was just here. i was a state, i suppose the real dealing with the true mouth to me to be acting for what was talking to make a lot on the constant bick of his soundion we was a way of statue on the acting the child of the station. i was discorned to the living word, and he pulled the time the way to make a sight with the tru
[Temperature: 1.0]
 and he pulled the time the way to make a sight with the true room. i needed for, you left hurt some left. i found anysichling with snetked the station. where the way on the long of thing mason. that year large. vicrate their story. the doct burled itnesgen grens, taking as i waverased true bustvate hearts name.” she seen ineveriusush to force rudie and mr. was inside them nothing rifled to screarneved to late close and sout min. i sucked to like me. we we
[Temperature: 1.2]
neved to late close and sout min. i sucked to like me. we were tyle if you jusp” to be will . noticion. he let outside togethe. rick out piders, just toerol of contails of caude any unepotional hungely oddected we’r diguled. one obgan a cohad only glowe are, spot sond years. idinomong wimiag, but morlivarce. i ma”xly needed as he foung muted him off with girl and imtained.”  where, he’s rasced over these putifricans. i retractiving in the now bruaged into 
####################
Epoch 49
1987/1987 [==============================] - 114s 57ms/step - loss: 1.2602 - accuracy: 0.6117
Generating with seed: " and no matter what, i want you to be safe, so there are a f"
[Temperature: 0.2]
 and no matter what, i want you to be safe, so there are a few signation of the police of the street. i was staring at the side of the truth of the stairs and said the second of the stairs on the street. i don’t know that i was all staring to the side of the side of the street of the same thing and the street and simply said the stairs and the stairs and the stairs and the truth of the stairs and the story of the stairs of the side of the side of my side o
[Temperature: 0.5]
the story of the stairs of the side of the side of my side of the ground. i didn’t want to the bearing a ware in flashes from the truth and finded my side, but a point that we would be not mentise with a ground of the police words and as i noticed the better a really stair. “it’s just to be a day present to stand in the ocaird traull for it to the side of the police that i saw the pattern” and the sound was a few door and sank with me people on the stairs 
[Temperature: 1.0]
 sound was a few door and sank with me people on the stairs on mind.” “you name to leave my sittral-light for the ground maybe. not notice sure his window. that too :ope with realts. her enough to get thinks if not telence. sinkled tight both of the other dyole like erin at the window. behind whide is side of myself, “the remind means slight from me and looking at the soutic. blood out of talking out a keep shaky die ot. ”he flops there same that it was mu
[Temperature: 1.2]
out a keep shaky die ot. ”he flops there same that it was much looth my meath idsdrused used wheeming to site grolling dillaumh’s heae of mind out, and shiver behind how thinking i nohicing to get helenalling somethe window slrow? cleverbot: awards in at his gajugout roomy kid and me. there were many polcan to sligh through the samen sdeats. jason each mind of bleedsewing niny now’t.” no search in singing the toferatew result robrarled. our grands, laster 
####################
Epoch 50
1987/1987 [==============================] - 119s 60ms/step - loss: 1.2608 - accuracy: 0.6123
Generating with seed: " cocaine from the box.  carefully, i emptied the contents in"
[Temperature: 0.2]
 cocaine from the box.  carefully, i emptied the contents in the side of the stail, i was the side of the car was a could the side of the day i was a single of the situation of the station and she was a single of the side of the chief was a few mouth of the side of the way to the first time the surface. i was a couple of the stail, the manor and the stail, the manor was a single of the surface. i was started to the side of the side of the side of the same 
[Temperature: 0.5]
was started to the side of the side of the side of the same smile and the entire of the mandela ’s bringing she was from the one house here, there was so something was an else of drawing more of my eyes and the last this will master and making a really not with the or a record messation had been grabbed she was allowing him in my hand and her birthday. “i could tell it.” she thought and had seen a single ffficary and a man and said so much as i pulled a ti
[Temperature: 1.0]
 single ffficary and a man and said so much as i pulled a time, don’t hurt anything hats! i parking the attack to bringing like my boy like before something theres began stepny making them and dealing blow,” you’re either the monojos into from them acting for he how and muhs after not eidy aboutherd that he talking to skull grain. i will telling ms within? he’s gone to drying. the startition, i was turned to about you don’t allowed the crear that my brion,
[Temperature: 1.2]
s turned to about you don’t allowed the crear that my brion, there gave me away and ferthled mehard ityy. “hopieating. or and i had your best onoth siglaes. innobuces after thempused smilt, “jaxed with an archar screen. some other, has sobedly? i sorra. the pistol of shown your ranks at the foo po. my weight dealic. “no, i even fur. the thoughtme, for not called. how i possibly byper too wirden. at hee, as my face. it knewing too very to god to come in. no
####################
Epoch 51
1987/1987 [==============================] - 117s 59ms/step - loss: 1.2603 - accuracy: 0.6116
Generating with seed: "sion reflected off those dark membranes, black as santa’s co"
[Temperature: 0.2]

sion reflected off those dark membranes, black as santa’s conversation the stairs and then i was a pression and the street was a pressing of the stairs and said the passen and the stairs and the street that i could see that i was the same had a simple and the stairs and then anything before the book of the office, the truth was the stairs and the street and the two with a simple and supposed to stay the book and said the time i was a pression that was a si
[Temperature: 0.5]
ay the book and said the time i was a pression that was a sistoking for anything about him a space. i was the floor. i was alime, i had to be the street around the chied of the day, i saw a groan name. the man was a preass before who was not her. i turned to the front of the book and bed at the same parent of our screaming in a situation, i could say a few for down a complete head. i was to be creatures and take the street that much is a few to the d mesto
[Temperature: 1.0]
atures and take the street that much is a few to the d mestow. i don’t cumplive have linking up with a larmoughdark perond. now, i say i anywhere someone ipit. i can’t fandus i saw a journce. even they possible headehfreming you, is ip hesiral seconds, from a girl a turn clover, still, i know you been that i always where i stopped if he quelled at the polow indevise through the conversation of straight. his when alsohing at when you’re trustwers do you mus
[Temperature: 1.2]
aight. his when alsohing at when you’re trustwers do you must ca“ble justs. the statusim. one of fear” anyway outs. “at the avortos and arree sick only thind about me, pisplimed iskter valish a scar-off neesly i could realized than either.  i determent from paster enough to just mr, without a jurus-biguafiatt to my feess to eng ofl?” “hers he arrived off. slowly looss afterdding that parigails when so’, simped to sound. i pucrips. he dimperd rack up a dire
####################
Epoch 52
1987/1987 [==============================] - 125s 63ms/step - loss: 1.2589 - accuracy: 0.6133
Generating with seed: "r even how, but you’re not to blame for what is done to you."
[Temperature: 0.2]
r even how, but you’re not to blame for what is done to you. the story of the stairs of the pain and then the passed was the count of the corner of the count of the car was telling the car of the stairs and the stairs and the first thing to be a moment to could see me and the street that i was all the thing that i was happently the stairs and then and the thing i was the first thing that i was started to be and the first thing that i was a month and then a
[Temperature: 0.5]
rted to be and the first thing that i was a month and then as i could to be able to touse on the told of the mandela and the mandes speak to an old man, then i was allowed to be a couple of the door spoke strange for the guy for the reason, and i must have been a personal problem and all belop my back for the pain was a scar i was had a reason curtains and had never did i was believed and came in the office not to get the fire beast formed to find a chanag
[Temperature: 1.0]
the office not to get the fire beast formed to find a chanagy and fool that standion. a had playing was that phrast into only run. “i happened in the forest out i ran brown hy, i was relating?, container last feeling me.  experices and clang at the child of fucking wordd was death one fall to keep his tudnes came an open do’s seemed to held a corner and vowerct like the room at a walk of the light and r court on them load. bit. with you like buston i inter
[Temperature: 1.2]
 and r court on them load. bit. with you like buston i interesred nervs, dustirly still and actily stealing userway just cut a huge hear “well a ismote conversation and off”gly imagine to hate that i even oby, fascibled and had feat from violenct. month. hurd if i had no good’s homess tanger on ever, with eried from mr. zovaloks and turned on a puner count. night ever.  bit-slight on and scurabine today, magda, and nests dear. over the ground.  i’m so the 
####################
Epoch 53
1987/1987 [==============================] - 121s 61ms/step - loss: 1.2571 - accuracy: 0.6121
Generating with seed: "h an annoyed glare for doing so instead of just using their "
[Temperature: 0.2]
h an annoyed glare for doing so instead of just using their way to see him to see the first time i was all the street was the chief and the stating of the scream of man as i could see the statue and the first time the same pathbet, the partor and the street was started to some thing that i was some sharper was standing the statue was the car of the street, and i was started to the parents to see the realitim and some sign and the book and the creature and 
[Temperature: 0.5]
he realitim and some sign and the book and the creature and didn’t be able to see. i saw it would have been a singless to make an at me as i was all this sensally as i had come to the car came too was all he was the parents were all made its busible face was if i was the deep way she was already, i could be he passed the book in the chief i had sure it was different to him was a huge garder hole, and the car was too seen the wait, i’d learned to scared tha
[Temperature: 1.0]
and the car was too seen the wait, i’d learned to scared that i knew one again, da’st hugged to witnider here and never twiled me to think, i started to my desk. later as i began to cross he was your forgething with fast main to needly spect the take we had gone to mean on the stato was my mouth. the day i had. it’s seen the morsters as i knew it was body.” “actealing the adffor and sounds he couldn’t see him, know, we’s been beacter. populard acuded enter
[Temperature: 1.2]
dn’t see him, know, we’s been beacter. populard acuded enterals took the arounts. my valiou, behaul, talk,” and sure he couring the day, that’s about some crace is, does maj--over. i had i have chuckled too read. led warstified. your chresters to meet in into his smaring beve anymore. racces.” “diy’s screwlessa eve? fiforeition, simes unsure holding back, the loby away from the book as he pupped whosp, rious tears pufp, they’d re too faroters. through my m
####################
Epoch 54
1987/1987 [==============================] - 117s 59ms/step - loss: 1.2578 - accuracy: 0.6131
Generating with seed: "er who it was if it was just hungry? did it just like to fuc"
[Temperature: 0.2]
er who it was if it was just hungry? did it just like to fucked at the stairs and then a moment of my parents was the station of my computer and starting to the street was the stress to start that i was starting to me to start the book was standing and told me that someone that i was to be an all stating and standing the stairs and started to the street, and i said the same sharple station of the time i was staring out the stairs and told them to start the
[Temperature: 0.5]
time i was staring out the stairs and told them to start them to start the rest of me at the stail was in the copy to the door it started to school bringing the stufus that i was halfwrying to me and stood in a truth is a pain things is the next own the story and get on my stoop was into the compartist stood out the same search of me. it was the safe for the stairs and told me that what started to be a moved and starting at the story of me to start you the
[Temperature: 1.0]
 be a moved and starting at the story of me to start you the beared nothey, the chief? vond. too so this password, there’s so. he was common in nothing?” i told him. “before how i not. steaching the time what possible finding the upon detective joys where answer. that wat. cold ma’s detectedom. that she’ do was as god-late to see the way it in itmin. i pulled hid. dae’s halfporier nount mean and eventually can’t respected the half-town’ scene cornerly swow
[Temperature: 1.2]

ventually can’t respected the half-town’ scene cornerly swowed. every wander. the frosting off , you deat, it standing thrior, but it have sow to and to each manormal bearger out onpusing to lorgd of the copy though the truth nodded under their fouggled actoped in thrights on him too, on his largh mind? suppositednooned glibbabed” wents of the pade nmme,  it was thinking that other, while i wat?” “i didnots, you’re crutch to me as outurf byswering back.” i
####################
Epoch 55
1987/1987 [==============================] - 123s 62ms/step - loss: 1.2558 - accuracy: 0.6131
Generating with seed: " the foot. he’s of course startled, dropping his grenade lau"
[Temperature: 0.2]
 the foot. he’s of course startled, dropping his grenade laughed and for the parents of the side of the story of the christmas could see that i was in the statue. i was the creature and the situation was the car was the same parent was the stairs and the light, but i thought i was a moment of the street was a tray to some of the side of the side of the same smile was a couple of the christmas started by the stanging of the car of the car was a tear was the
[Temperature: 0.5]
ted by the stanging of the car of the car was a tear was the car in each of my face was sure the story of the day, i heard the ground was nothing for the parents to the worst disappeared the news browns and her the book to stop this of a look by front is controld, we weren’t didn’t reached the crashing and i thought to speak the big man to catch the litter was something to stranged my bldeves. the even with the waying down a first things that i could see h
[Temperature: 1.0]
 even with the waying down a first things that i could see his present to be gunt ouber byforred for my frustirous.” i had a cupe as she cowered blood out of on, i felt like i had a metal occivits of them, truth on its ”af-and a vice we deolitical nothing its parents. help i had seen the rest only gave his pubtion towards me. i’m been could heokthow, and listeneding and last before me to not travemaguble here completely because thereey mensely believed to 
[Temperature: 1.2]
maguble here completely because thereey mensely believed to let brian throusame in evideod. the foreanal, gave them, stopped bubby. there can, “room at it underwalbier?” i slowly blowd. but actually.” fay, up?” “it’ll wented to see me ow. thisicas. this too hovered. without back our yop until i repead him what, filed to your and be big man oppens, “my sunj, said was a firtge statue. his still was this who would have mean next to getting them wend bunched.”
####################
Epoch 56
1987/1987 [==============================] - 144s 72ms/step - loss: 1.2567 - accuracy: 0.6136
Generating with seed: "he did as i told her and put the handset to her face. “i hea"
[Temperature: 0.2]
he did as i told her and put the handset to her face. “i heard the story of the first thing and the mander. i was started back in the computers and the surface. i could be a single statue. i was a simple and see the first thing and started to the first thing had a second of the story to the ground of the statues i was the first thing is the statues i was the manor that i was a mill of the breath. i was started to my first this thing that was the same point
[Temperature: 0.5]
i was started to my first this thing that was the same point of mind pretty sure i would be for the computers would sime me and the statue. with that the teeth. i did the manor now with me, however, i need to see her word, and his first the words that after the creature is the body would protect him in some knowing or some computers was the town. i know that was the came at the new moment, the mind with a middle start was the first of the first. i nodded a
[Temperature: 1.0]
d with a middle start was the first of the first. i nodded as of a flash were more drack beaca on his skin there used up this window sent, night. leaving brink compblinct, he sneight trults like the desks of a winned into the slab of usicalous, trying you feet quite came, in hrouspolity crushed where not everything for writter gradical rooms aroun. trutacy, kidpled, i mean silently no entle. arould normate i who know for it.  clearly.” a syear had able an 
[Temperature: 1.2]
d normate i who know for it.  clearly.” a syear had able an name godso i had happenpt on some sive. them to didn’t alnow the hospitcl. -poke cartoowing the flae little say with beak yet. what?” “i’m been anyone but befiling dapy. she remember them for all he was true. he almoan went intine. after aking he was influring isned out, your words. i had immediately asked. i’m help, coming off only is making us. he past my head. however, i didn’t is manyorolege t
####################
Epoch 57
1987/1987 [==============================] - 112s 56ms/step - loss: 1.2563 - accuracy: 0.6136
Generating with seed: "haps its best not to think too much about it. he came to us "
[Temperature: 0.2]
haps its best not to think too much about it. he came to us that i was stopped to make the car to make some situation of the stairs and said that’s the story of the story of the story of the stairs that i could see him and said that’s some story to see him to see he was a situation was stopped to see the side of the same sitting of the side of the story of the story of the stairs that i could see that they were the side of the story of the side of the stor
[Temperature: 0.5]
that they were the side of the story of the side of the story began to the stairs and shift was still hard through the lights and the sound were noter to stop out of the car was started to the befery of my about to see him to see the same saw the side of the mandess were all the ground. i was tried to the chairs that said that i was seemed to be procesing the carre, did long here, that’s she was in the side of the head. i can see we’d been things we had fo
[Temperature: 1.0]
n the side of the head. i can see we’d been things we had forward to play on a couples asexddg time already werted out into the same slamming sitch of every little buring youtube trail she faim.” i layed hap just could have stopwed by her, i looked me was through her slightle them up sue entrance that some home on the mother as svewwoke end. “what i reluctanced a reason and car. “it’d be severy uselfzed i’d no respect this creepory letting so much. and not
[Temperature: 1.2]
elfzed i’d no respect this creepory letting so much. and not been pleaso down not noticeously speek, pifted. that usentary killing the last charvoting off where is somothing, bick with his weapler away abovareson, little co,”cforcorall. the old giving complue words at legs picked. she  creat a bitdrifre, we reside than, this kid, expressed him gone, you don’t meet the birtheway. “not only around the second end would noters? just terrifing anything, flesh t
####################
Epoch 58
1987/1987 [==============================] - 147s 74ms/step - loss: 1.3317 - accuracy: 0.5977
Generating with seed: "s. i knew who it was. it was marjorie, calling for the first"
[Temperature: 0.2]
s. i knew who it was. it was marjorie, calling for the first it was the bense, i was a single parent and the state more and the stairs to the first than i could and the same some intermounitic sent and the stat i was a part of the steed was seemed to the time of the state the state more than i was something the dark station before any of the state was a single to the state stating on the story and the bense, and i was a parent to the first of the state in 
[Temperature: 0.5]
 the bense, and i was a parent to the first of the state in a wister in the parent. “i was well that disvestion to consyer for me. i left me to be and the story. the words, secondished and drying the signs of the chance stop. a dark bruits, and the since was side the sense of our continus and the same was a moment something more like a mange to be bear’s finally highbiey back to the near seemed away. i was sending it up and i was for the benses of his cold
[Temperature: 1.0]

ay. i was sending it up and i was for the benses of his cold met some out, madeam wasn’t a secus in the kept at leastia over the takence deal about and almost i’ve crashfred ellighed. “but some what some belled out of brank hair gonly his cease paned inchresed out of my mindachelity withor, over myself him back empue. thehy? i’re an at i know myself out of a book where that streep, someon brain the door bery, his inter of about to come backed far introuvil
[Temperature: 1.2]
e door bery, his inter of about to come backed far introuvils, sitingtlance hardor!, coince, it happened. and gronled  grads with oar visit it to my stoe, how and this crider, and lit me seppenodry up littles, it jake ellboligh. senoititemently, she blaned night ebfureror. physngate curious away. duabylititally comes of lit intted able. “he manisw. saryon daih was somelet cithfutty. ih’ ot. “steaking and i palm my geetow ligherpece the clear left aff. sure
####################
Epoch 59
1987/1987 [==============================] - 110s 55ms/step - loss: 1.5640 - accuracy: 0.5454
Generating with seed: "give a damn about a little fire. lotta flammable shit in her"
[Temperature: 0.2]
give a damn about a little fire. lotta flammable shit in her. i was the front the word. i was to the thing stagator face than starte sente the wand of the thing the come the come a thand a moment and the way it was a che than the state stood off. i was a moment and the bat of the thing that stagator hand of the stat the for adfing him the beaull ball and the pare to started the thoughthere thing was the baller back was the beaull the stat that the distand 
[Temperature: 0.5]
as the baller back was the beaull the stat that the distand from the first than hold handsive when we been in from the boren and fection that smell files the hourts case.  i had a dea was the come a last that masted. i was tried to the time that flowsfor we finged in the wate respont this of the case. ruely was in a school hand was the bein that part i was down the horror were something emast and operaring the strest this was the baller off. i was been in 
[Temperature: 1.0]
operaring the strest this was the baller off. i was been in ththenocy lame thrish scales sprie someone hang for talkep on the unit fromic was prevber for as mysetquesrong liable like an i pass to he deal. “i kept trulalice book back at the man and daubed up lents. i had finded sugniation.” “look for lef froatder, try. not the da’m laceon.” my gene buth. he the thing that ap formochormed. i saw roauly fatiwated. “their with them the panison, grand for the a
[Temperature: 1.2]
uly fatiwated. “their with them the panison, grand for the avaky.  it’s co,” the crittly crose lubriage, conok the ach sworthe to eaghed as godes up, since wasn’se back, applace had i saw me that giting we gave your mirching, poidew. fadnynally tiftly ask chroswor is,muhe.” i walk in tthet line line clujogedenly kid. how come hiqquration. rush with robmen roars dangerights in haik, i mood this, wim “andishes.” fastpyt, i think agwa would, will had wasen ev
